<a href="https://colab.research.google.com/github/lynnfdsouza/fiber_system/blob/main/Fibre_System1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters
            min_cable_length (float): Minimum cable length to consider for detection
        """
        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = []
        self.operator_locations: List[Tuple[float, float]] = []
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)  # Clustering for corridor detection

    def simulate_sensor_data(self, num_cables: int = 10) -> List[Dict]:
        """
        Simulate laser/acoustic sensor data for fibre optic cables.

        Returns:
            List of dictionaries containing cable coordinates and metadata
        """
        cables = []
        for _ in range(num_cables):
            # Simulate cable trajectory as 3D coordinates (x, y, z)
            start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
            end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
            cable = {
                'start': start_point,
                'end': end_point,
                'length': np.linalg.norm(np.array(end_point) - np.array(start_point)),
                'timestamp': time.time(),
                'is_active': np.random.choice([True, False], p=[0.8, 0.2])  # 80% chance cable is active
            }
            if cable['length'] >= self.min_cable_length:
                cables.append(cable)
        logger.info(f"Simulated {len(cables)} cable detections")
        return cables

    def detect_cables(self) -> None:
        """
        Detect fibre optic cables in real-time using simulated sensor data.
        """
        new_cables = self.simulate_sensor_data()
        for cable in new_cables:
            if cable['length'] >= self.min_cable_length and cable not in self.detected_cables:
                self.detected_cables.append(cable)
                logger.info(f"Detected cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")

    def trace_operator(self, cable: Dict) -> Tuple[float, float]:
        """
        Trace cable back to estimated operator location.

        Args:
            cable (Dict): Detected cable data with start and end points

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator
        """
        # Assume operator is at the start point (simplified model)
        operator_location = (cable['start'][0], cable['start'][1])
        self.operator_locations.append(operator_location)
        logger.info(f"Traced operator to location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering to identify preferred drone corridors based on cable trajectories.

        Returns:
            np.ndarray: Cluster labels for detected cables
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract start and end points for clustering
        points = np.array([c['start'][:2] + c['end'][:2] for c in self.detected_cables])
        if len(points) < 3:
            logger.warning("Insufficient data for corridor clustering")
            return np.array([])

        # Perform DBSCAN clustering to identify corridors
        clusters = self.corridor_model.fit_predict(points)
        logger.info(f"Identified {len(set(clusters)) - (1 if -1 in clusters else 0)} corridors")
        return clusters

    def integrate_with_defense_stack(self) -> Dict:
        """
        Simulate integration with a layered defense stack by outputting detection data.

        Returns:
            Dict: Structured data for defense stack integration
        """
        defense_data = {
            'timestamp': time.time(),
            'detected_cables': self.detected_cables,
            'operator_locations': self.operator_locations,
            'corridor_clusters': self.predict_corridors().tolist()
        }
        logger.info("Generated defense stack integration data")
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.

        Args:
            filename (str): Output file name
        """
        # Convert boolean values to strings for JSON serialization
        serializable_data = self.integrate_with_defense_stack()
        for cable in serializable_data['detected_cables']:
            cable['is_active'] = str(cable['is_active'])

        with open(filename, 'w') as f:
            json.dump(serializable_data, f, indent=2)
        logger.info(f"Saved detection data to {filename}")

    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation.

        Args:
            iterations (int): Number of detection cycles
            interval (float): Time interval between detections in seconds
        """
        logger.info("Starting FIBERHOUND system simulation")
        for i in range(iterations):
            self.detect_cables()
            for cable in self.detected_cables[-10:]:  # Process last 10 cables
                if cable['is_active']:
                    self.trace_operator(cable)
            self.save_data()
            logger.info(f"Iteration {i+1}/{iterations} completed")
            time.sleep(interval)
        logger.info("FIBERHOUND simulation completed")

def main():
    # Initialize and run FIBERHOUND system
    fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
    fiberhound.run(iterations=5, interval=1.0)

if __name__ == "__main__":
    main()

# Task
Refactor the provided Python script for a cable detection system to use real sensor data instead of simulated data, enhance trajectory analysis in the `trace_operator` method, outline integration with real-time laser hardware, and expand the `integrate_with_defense_stack` method with placeholder API endpoints or communication protocols.

## Replace simulated data

### Subtask:
Modify the `simulate_sensor_data` method to accept real sensor data as input.


**Reasoning**:
The subtask requires modifying the `simulate_sensor_data` method to accept real sensor data. This involves renaming the method, changing its signature to accept input data, removing the simulation logic, and adding processing logic for the new input format. I will implement these changes in a single code block.



In [ ]:
class FiberHound:
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters
            min_cable_length (float): Minimum cable length to consider for detection
        """
        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = []
        self.operator_locations: List[Tuple[float, float]] = []
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)  # Clustering for corridor detection

    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except KeyError as e:
                logger.warning(f"Skipping sensor data point due to missing key: {e}")
            except Exception as e:
                logger.error(f"Error processing sensor data point: {e}")

        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        for cable in new_cables:
            if cable not in self.detected_cables:
                self.detected_cables.append(cable)
                logger.info(f"Detected cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")

    def trace_operator(self, cable: Dict) -> Tuple[float, float]:
        """
        Trace cable back to estimated operator location.

        Args:
            cable (Dict): Detected cable data with start and end points

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator
        """
        # Assume operator is at the start point (simplified model)
        operator_location = (cable['start'][0], cable['start'][1])
        self.operator_locations.append(operator_location)
        logger.info(f"Traced operator to location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering to identify preferred drone corridors based on cable trajectories.

        Returns:
            np.ndarray: Cluster labels for detected cables
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract start and end points for clustering
        points = np.array([c['start'][:2] + c['end'][:2] for c in self.detected_cables])
        if len(points) < 3:
            logger.warning("Insufficient data for corridor clustering")
            return np.array([])

        # Perform DBSCAN clustering to identify corridors
        clusters = self.corridor_model.fit_predict(points)
        logger.info(f"Identified {len(set(clusters)) - (1 if -1 in clusters else 0)} corridors")
        return clusters

    def integrate_with_defense_stack(self) -> Dict:
        """
        Simulate integration with a layered defense stack by outputting detection data.
        Includes placeholder API endpoints or communication protocols.

        Returns:
            Dict: Structured data for defense stack integration
        """
        defense_data = {
            'timestamp': time.time(),
            'detected_cables': self.detected_cables,
            'operator_locations': self.operator_locations,
            'corridor_clusters': self.predict_corridors().tolist(),
            'integration_endpoints': {
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                'corridor_analysis_service': 'AMQP://message.queue.com/corridor_analysis'
            }
        }
        logger.info("Generated defense stack integration data with placeholder endpoints")
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.

        Args:
            filename (str): Output file name
        """
        # Convert boolean values to strings for JSON serialization
        serializable_data = self.integrate_with_defense_stack()
        for cable in serializable_data['detected_cables']:
            cable['is_active'] = str(cable['is_active'])

        with open(filename, 'w') as f:
            json.dump(serializable_data, f, indent=2)
        logger.info(f"Saved detection data to {filename}")

    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation.

        Args:
            iterations (int): Number of detection cycles
            interval (float): Time interval between detections in seconds
        """
        logger.info("Starting FIBERHOUND system simulation")
        # In a real scenario, this loop would read from a sensor stream
        # For this simulation, we'll generate some dummy sensor-like data
        for i in range(iterations):
            logger.info(f"Simulating sensor data input for iteration {i+1}")
            # Simulate receiving sensor data - replace with actual sensor reading
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(5, 15) # Simulate variable sensor input
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 dummy_sensor_data.append({
                     'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                     'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                     'timestamp': time.time(),
                     'is_active': np.random.choice([True, False], p=[0.8, 0.2])
                 })

            self.detect_cables(dummy_sensor_data) # Pass the simulated sensor data

            # Process only newly added cables or a recent batch
            # This is a simplification; real system would process based on timestamps/IDs
            recent_cables_to_process = self.detected_cables[-len(dummy_sensor_data):] if len(dummy_sensor_data) > 0 else []
            for cable in recent_cables_to_process:
                 if cable['is_active']:
                     self.trace_operator(cable)

            self.save_data()
            logger.info(f"Iteration {i+1}/{iterations} completed")
            time.sleep(interval)
        logger.info("FIBERHOUND simulation completed")

def main():
    # Initialize and run FIBERHOUND system
    fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
    fiberhound.run(iterations=5, interval=1.0)

if __name__ == "__main__":
    main()

## Enhance operator tracing

### Subtask:
Improve the `trace_operator` method to use more sophisticated trajectory analysis techniques.


**Reasoning**:
Modify the `trace_operator` method to implement a more sophisticated operator location estimation logic, moving away from the simple assumption that the operator is at the start point of the cable. This involves analyzing the sequence and properties of detected cables.



In [ ]:
class FiberHound:
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters
            min_cable_length (float): Minimum cable length to consider for detection
        """
        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)  # Clustering for corridor detection
        # Store historical cable data for trajectory analysis
        self.cable_history: List[Dict] = []

    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except KeyError as e:
                logger.warning(f"Skipping sensor data point due to missing key: {e}")
            except Exception as e:
                logger.error(f"Error processing sensor data point: {e}")

        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        for cable in new_cables:
            if cable not in self.detected_cables:
                self.detected_cables.append(cable)
                # Add to history for trajectory analysis
                self.cable_history.append(cable)
                logger.info(f"Detected cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")

    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data.

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
        """
        logger.info(f"Tracing operator for recent cable: start={recent_cable['start']}, end={recent_cable['end']}")

        # Implement sophisticated trajectory analysis
        # This is a placeholder logic. A real implementation would involve:
        # 1. Analyzing the temporal sequence of cable detections (self.cable_history)
        # 2. Identifying connected or nearby cable segments
        # 3. Estimating laying direction and speed
        # 4. Potentially using Kalman filters or other tracking algorithms

        # Placeholder Logic:
        # - Find the cable in history that most recently precedes the current one.
        # - Assume the operator is likely moving in the direction from the end of the previous cable
        #   segment towards the start of the current segment.
        # - Estimate the operator location as a point on the line extending backward from the start
        #   of the recent cable, influenced by the direction of the previous segment.
        # - If no previous cable is found, default to the start of the current cable (like the old logic).

        previous_cables = [c for c in self.cable_history if c['timestamp'] < recent_cable['timestamp']]
        previous_cables.sort(key=lambda x: x['timestamp'], reverse=True) # Sort by timestamp descending

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point

        if previous_cables:
            most_recent_previous_cable = previous_cables[0]
            prev_end = np.array(most_recent_previous_cable['end'][:2])
            curr_start = np.array(recent_cable['start'][:2])

            # Calculate the direction vector from the end of the previous cable to the start of the current one
            direction_vector = curr_start - prev_end

            # Normalize the direction vector
            norm = np.linalg.norm(direction_vector)
            if norm > 0:
                normalized_direction = direction_vector / norm

                # Estimate operator location by extending backward from the current start point
                # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                # in the opposite direction of the estimated laying direction.
                # This is a simplification; a real model would be more complex.
                estimated_distance_behind_start = np.mean([c['length'] for c in self.cable_history]) * 0.5 if self.cable_history else 100.0 # Use average length or default
                estimated_operator_pos_2d = curr_start - normalized_direction * estimated_distance_behind_start

                operator_location = (estimated_operator_pos_2d[0], estimated_operator_pos_2d[1])
                logger.info(f"Estimated operator location based on trajectory analysis with previous cable (timestamp: {most_recent_previous_cable['timestamp']:.2f})")
            else:
                 logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
        else:
            logger.info("No previous cable history found for trajectory analysis, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering to identify preferred drone corridors based on cable trajectories.

        Returns:
            np.ndarray: Cluster labels for detected cables
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract start and end points for clustering
        points = np.array([c['start'][:2] + c['end'][:2] for c in self.detected_cables])
        if len(points) < 3:
            logger.warning("Insufficient data for corridor clustering")
            return np.array([])

        # Perform DBSCAN clustering to identify corridors
        clusters = self.corridor_model.fit_predict(points)
        logger.info(f"Identified {len(set(clusters)) - (1 if -1 in clusters else 0)} corridors")
        return clusters

    def integrate_with_defense_stack(self) -> Dict:
        """
        Simulate integration with a layered defense stack by outputting detection data.
        Includes placeholder API endpoints or communication protocols.

        Returns:
            Dict: Structured data for defense stack integration
        """
        defense_data = {
            'timestamp': time.time(),
            'detected_cables': self.detected_cables,
            'operator_locations': self.operator_locations,
            'corridor_clusters': self.predict_corridors().tolist(),
            'integration_endpoints': {
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                'corridor_analysis_service': 'AMQP://message.queue.com/corridor_analysis'
            }
        }
        logger.info("Generated defense stack integration data with placeholder endpoints")
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.

        Args:
            filename (str): Output file name
        """
        # Convert boolean values to strings for JSON serialization
        serializable_data = self.integrate_with_defense_stack()
        for cable in serializable_data['detected_cables']:
            cable['is_active'] = str(cable['is_active'])

        with open(filename, 'w') as f:
            json.dump(serializable_data, f, indent=2)
        logger.info(f"Saved detection data to {filename}")

    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation.

        Args:
            iterations (int): Number of detection cycles
            interval (float): Time interval between detections in seconds
        """
        logger.info("Starting FIBERHOUND system simulation")
        # In a real scenario, this loop would read from a sensor stream
        # For this simulation, we'll generate some dummy sensor-like data
        for i in range(iterations):
            logger.info(f"Simulating sensor data input for iteration {i+1}")
            # Simulate receiving sensor data - replace with actual sensor reading
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(5, 15) # Simulate variable sensor input
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 dummy_sensor_data.append({
                     'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                     'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                     'timestamp': time.time(),
                     'is_active': np.random.choice([True, False], p=[0.8, 0.2])
                 })

            self.detect_cables(dummy_sensor_data) # Pass the simulated sensor data

            # Process only newly added cables or a recent batch
            # This is a simplification; real system would process based on timestamps/IDs
            recent_cables_to_process = self.detected_cables[-len(dummy_sensor_data):] if len(dummy_sensor_data) > 0 else []
            for cable in recent_cables_to_process:
                 if cable['is_active']:
                     self.trace_operator(cable)

            self.save_data()
            logger.info(f"Iteration {i+1}/{iterations} completed")
            time.sleep(interval)
        logger.info("FIBERHOUND simulation completed")

def main():
    # Initialize and run FIBERHOUND system
    fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
    fiberhound.run(iterations=5, interval=1.0)

if __name__ == "__main__":
    main()

## Integrate with hardware

### Subtask:
Outline the steps required to integrate the system with real-time hardware for laser-based cable detection.


**Reasoning**:
Add methods for hardware initialization and reading, and modify the run method to use these new methods to fulfill the subtask of outlining hardware integration.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters
            min_cable_length (float): Minimum cable length to consider for detection
        """
        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)  # Clustering for corridor detection
        # Store historical cable data for trajectory analysis
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        # Placeholder: Outline steps for real hardware integration
        # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
        # 2. Load necessary drivers or libraries for the specific sensor model.
        # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
        # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
        # 5. Perform self-tests or calibration procedures if available.
        # 6. Handle potential connection errors or hardware failures.

        # Simulate initialization success
        time.sleep(1) # Simulate time taken for initialization
        self.hardware_initialized = True
        logger.info("Sensor hardware initialization completed (simulated).")
        return self.hardware_initialized

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        # Placeholder: Outline steps for reading real hardware data
        # 1. Read data from the established hardware connection stream.
        # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols).
        # 3. Manage buffering of incoming data.
        # 4. Apply sampling rates or data filtering as needed.
        # 5. Implement error handling for data corruption, connection loss, or hardware read errors.
        # 6. Convert raw data into a structured format (e.g., list of dictionaries)
        #    that can be processed by `process_sensor_data`. The format should match
        #    the expected input of `process_sensor_data`:
        #    {'start_x': float, 'start_y': float, 'start_z': float,
        #     'end_x': float, 'end_y': float, 'end_z': float,
        #     'timestamp': float, 'is_active': bool}

        # Simulate reading dummy sensor data for now
        dummy_sensor_data = []
        num_simulated_detections = np.random.randint(5, 15) # Simulate variable sensor input
        for _ in range(num_simulated_detections):
             start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
             end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
             dummy_sensor_data.append({
                 'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                 'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                 'timestamp': time.time(),
                 'is_active': np.random.choice([True, False], p=[0.8, 0.2])
             })
        logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated).")
        return dummy_sensor_data


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except KeyError as e:
                logger.warning(f"Skipping sensor data point due to missing key: {e}")
            except Exception as e:
                logger.error(f"Error processing sensor data point: {e}")

        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        for cable in new_cables:
            if cable not in self.detected_cables:
                self.detected_cables.append(cable)
                # Add to history for trajectory analysis
                self.cable_history.append(cable)
                logger.info(f"Detected cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")

    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data.

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
        """
        logger.info(f"Tracing operator for recent cable: start={recent_cable['start']}, end={recent_cable['end']}")

        # Implement sophisticated trajectory analysis
        # This is a placeholder logic. A real implementation would involve:
        # 1. Analyzing the temporal sequence of cable detections (self.cable_history)
        # 2. Identifying connected or nearby cable segments
        # 3. Estimating laying direction and speed
        # 4. Potentially using Kalman filters or other tracking algorithms

        # Placeholder Logic:
        # - Find the cable in history that most recently precedes the current one.
        # - Assume the operator is likely moving in the direction from the end of the previous cable
        #   segment towards the start of the current segment.
        # - Estimate the operator location as a point on the line extending backward from the start
        #   of the recent cable, influenced by the direction of the previous segment.
        # - If no previous cable is found, default to the start of the current cable (like the old logic).

        previous_cables = [c for c in self.cable_history if c['timestamp'] < recent_cable['timestamp']]
        previous_cables.sort(key=lambda x: x['timestamp'], reverse=True) # Sort by timestamp descending

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point

        if previous_cables:
            most_recent_previous_cable = previous_cables[0]
            prev_end = np.array(most_recent_previous_cable['end'][:2])
            curr_start = np.array(recent_cable['start'][:2])

            # Calculate the direction vector from the end of the previous cable to the start of the current one
            direction_vector = curr_start - prev_end

            # Normalize the direction vector
            norm = np.linalg.norm(direction_vector)
            if norm > 0:
                normalized_direction = direction_vector / norm

                # Estimate operator location by extending backward from the current start point
                # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                # in the opposite direction of the estimated laying direction.
                # This is a simplification; a real model would be more complex.
                estimated_distance_behind_start = np.mean([c['length'] for c in self.cable_history]) * 0.5 if self.cable_history else 100.0 # Use average length or default
                estimated_operator_pos_2d = curr_start - normalized_direction * estimated_distance_behind_start

                operator_location = (estimated_operator_pos_2d[0], estimated_operator_pos_2d[1])
                logger.info(f"Estimated operator location based on trajectory analysis with previous cable (timestamp: {most_recent_previous_cable['timestamp']:.2f})")
            else:
                 logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
        else:
            logger.info("No previous cable history found for trajectory analysis, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering to identify preferred drone corridors based on cable trajectories.

        Returns:
            np.ndarray: Cluster labels for detected cables
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract start and end points for clustering
        points = np.array([c['start'][:2] + c['end'][:2] for c in self.detected_cables])
        if len(points) < 3:
            logger.warning("Insufficient data for corridor clustering")
            return np.array([])

        # Perform DBSCAN clustering to identify corridors
        clusters = self.corridor_model.fit_predict(points)
        logger.info(f"Identified {len(set(clusters)) - (1 if -1 in clusters else 0)} corridors")
        return clusters

    def integrate_with_defense_stack(self) -> Dict:
        """
        Simulate integration with a layered defense stack by outputting detection data.
        Includes placeholder API endpoints or communication protocols.

        Returns:
            Dict: Structured data for defense stack integration
        """
        defense_data = {
            'timestamp': time.time(),
            'detected_cables': self.detected_cables,
            'operator_locations': self.operator_locations,
            'corridor_clusters': self.predict_corridors().tolist(),
            'integration_endpoints': {
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                'corridor_analysis_service': 'AMQP://message.queue.com/corridor_analysis'
            }
        }
        logger.info("Generated defense stack integration data with placeholder endpoints")
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.

        Args:
            filename (str): Output file name
        """
        # Convert boolean values to strings for JSON serialization
        serializable_data = self.integrate_with_defense_stack()
        for cable in serializable_data['detected_cables']:
            cable['is_active'] = str(cable['is_active'])

        with open(filename, 'w') as f:
            json.dump(serializable_data, f, indent=2)
        logger.info(f"Saved detection data to {filename}")

    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles
            interval (float): Time interval between detections in seconds
        """
        logger.info("Starting FIBERHOUND system simulation with hardware integration outline")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"Processing iteration {i+1}")

            # Step 2: Read data from hardware instead of simulating
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                self.detect_cables(raw_sensor_data) # Pass the data read from hardware

                # Process only newly added cables from this iteration's read
                # This is a simplification; real system would process based on timestamps/IDs
                recent_cables_to_process = self.detected_cables[-len(raw_sensor_data):] if len(raw_sensor_data) > 0 else []
                for cable in recent_cables_to_process:
                     if cable['is_active']:
                         self.trace_operator(cable)
            else:
                logger.warning(f"No data read from hardware in iteration {i+1}")


            self.save_data()
            logger.info(f"Iteration {i+1}/{iterations} completed")
            time.sleep(interval)

        logger.info("FIBERHOUND simulation completed")

def main():
    # Initialize and run FIBERHOUND system
    fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
    fiberhound.run(iterations=5, interval=1.0)

if __name__ == "__main__":
    main()

## Expand defense stack integration

### Subtask:
Modify the `integrate_with_defense_stack` method to include actual API endpoints or communication protocols for sending detection data to a defense stack.


**Reasoning**:
Modify the `integrate_with_defense_stack` method to include more realistic placeholder API endpoints and communication protocols with comments explaining their intended use.



In [ ]:
class FiberHound:
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters
            min_cable_length (float): Minimum cable length to consider for detection
        """
        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)  # Clustering for corridor detection
        # Store historical cable data for trajectory analysis
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        # Placeholder: Outline steps for real hardware integration
        # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
        # 2. Load necessary drivers or libraries for the specific sensor model.
        # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
        # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
        # 5. Perform self-tests or calibration procedures if available.
        # 6. Handle potential connection errors or hardware failures.

        # Simulate initialization success
        time.sleep(1) # Simulate time taken for initialization
        self.hardware_initialized = True
        logger.info("Sensor hardware initialization completed (simulated).")
        return self.hardware_initialized

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        # Placeholder: Outline steps for reading real hardware data
        # 1. Read data from the established hardware connection stream.
        # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols).
        # 3. Manage buffering of incoming data.
        # 4. Apply sampling rates or data filtering as needed.
        # 5. Implement error handling for data corruption, connection loss, or hardware read errors.
        # 6. Convert raw data into a structured format (e.g., list of dictionaries)
        #    that can be processed by `process_sensor_data`. The format should match
        #    the expected input of `process_sensor_data`:
        #    {'start_x': float, 'start_y': float, 'start_z': float,
        #     'end_x': float, 'end_y': float, 'end_z': float,
        #     'timestamp': float, 'is_active': bool}

        # Simulate reading dummy sensor data for now
        dummy_sensor_data = []
        num_simulated_detections = np.random.randint(5, 15) # Simulate variable sensor input
        for _ in range(num_simulated_detections):
             start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
             end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
             dummy_sensor_data.append({
                 'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                 'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                 'timestamp': time.time(),
                 'is_active': np.random.choice([True, False], p=[0.8, 0.2])
             })
        logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated).")
        return dummy_sensor_data


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except KeyError as e:
                logger.warning(f"Skipping sensor data point due to missing key: {e}")
            except Exception as e:
                logger.error(f"Error processing sensor data point: {e}")

        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        for cable in new_cables:
            if cable not in self.detected_cables:
                self.detected_cables.append(cable)
                # Add to history for trajectory analysis
                self.cable_history.append(cable)
                logger.info(f"Detected cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")

    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data.

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
        """
        logger.info(f"Tracing operator for recent cable: start={recent_cable['start']}, end={recent_cable['end']}")

        # Implement sophisticated trajectory analysis
        # This is a placeholder logic. A real implementation would involve:
        # 1. Analyzing the temporal sequence of cable detections (self.cable_history)
        # 2. Identifying connected or nearby cable segments
        # 3. Estimating laying direction and speed
        # 4. Potentially using Kalman filters or other tracking algorithms

        # Placeholder Logic:
        # - Find the cable in history that most recently precedes the current one.
        # - Assume the operator is likely moving in the direction from the end of the previous cable
        #   segment towards the start of the current segment.
        # - Estimate the operator location as a point on the line extending backward from the start
        #   of the recent cable, influenced by the direction of the previous segment.
        # - If no previous cable is found, default to the start of the current cable (like the old logic).

        previous_cables = [c for c in self.cable_history if c['timestamp'] < recent_cable['timestamp']]
        previous_cables.sort(key=lambda x: x['timestamp'], reverse=True) # Sort by timestamp descending

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point

        if previous_cables:
            most_recent_previous_cable = previous_cables[0]
            prev_end = np.array(most_recent_previous_cable['end'][:2])
            curr_start = np.array(recent_cable['start'][:2])

            # Calculate the direction vector from the end of the previous cable to the start of the current one
            direction_vector = curr_start - prev_end

            # Normalize the direction vector
            norm = np.linalg.norm(direction_vector)
            if norm > 0:
                normalized_direction = direction_vector / norm

                # Estimate operator location by extending backward from the current start point
                # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                # in the opposite direction of the estimated laying direction.
                # This is a simplification; a real model would be more complex.
                estimated_distance_behind_start = np.mean([c['length'] for c in self.cable_history]) * 0.5 if self.cable_history else 100.0 # Use average length or default
                estimated_operator_pos_2d = curr_start - normalized_direction * estimated_distance_behind_start

                operator_location = (estimated_operator_pos_2d[0], estimated_operator_pos_2d[1])
                logger.info(f"Estimated operator location based on trajectory analysis with previous cable (timestamp: {most_recent_previous_cable['timestamp']:.2f})")
            else:
                 logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
        else:
            logger.info("No previous cable history found for trajectory analysis, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering to identify preferred drone corridors based on cable trajectories.

        Returns:
            np.ndarray: Cluster labels for detected cables
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract start and end points for clustering
        points = np.array([c['start'][:2] + c['end'][:2] for c in self.detected_cables])
        if len(points) < 3:
            logger.warning("Insufficient data for corridor clustering")
            return np.array([])

        # Perform DBSCAN clustering to identify corridors
        clusters = self.corridor_model.fit_predict(points)
        logger.info(f"Identified {len(set(clusters)) - (1 if -1 in clusters else 0)} corridors")
        return clusters

    def integrate_with_defense_stack(self) -> Dict:
        """
        Simulate integration with a layered defense stack by outputting detection data.
        Includes placeholder API endpoints or communication protocols.

        Returns:
            Dict: Structured data for defense stack integration
        """
        defense_data = {
            'timestamp': time.time(),
            'detected_cables': self.detected_cables,
            'operator_locations': self.operator_locations,
            'corridor_clusters': self.predict_corridors().tolist(),
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST with JSON payload
                # UDP endpoint for sending real-time operator location updates
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP, Data Format: Binary or JSON
                # Message Queue topic for publishing corridor analysis results
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP, Data Format: JSON or Protobuf
                # Secure HTTP endpoint for system health and status
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT with JSON payload
            }
        }
        logger.info("Generated defense stack integration data with placeholder endpoints")
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.

        Args:
            filename (str): Output file name
        """
        # Convert boolean values to strings for JSON serialization
        serializable_data = self.integrate_with_defense_stack()
        for cable in serializable_data['detected_cables']:
            cable['is_active'] = str(cable['is_active'])

        with open(filename, 'w') as f:
            json.dump(serializable_data, f, indent=2)
        logger.info(f"Saved detection data to {filename}")

    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles
            interval (float): Time interval between detections in seconds
        """
        logger.info("Starting FIBERHOUND system simulation with hardware integration outline")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"Processing iteration {i+1}")

            # Step 2: Read data from hardware instead of simulating
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                self.detect_cables(raw_sensor_data) # Pass the data read from hardware

                # Process only newly added cables from this iteration's read
                # This is a simplification; real system would process based on timestamps/IDs
                recent_cables_to_process = self.detected_cables[-len(raw_sensor_data):] if len(raw_sensor_data) > 0 else []
                for cable in recent_cables_to_process:
                     if cable['is_active']:
                         self.trace_operator(cable)
            else:
                logger.warning(f"No data read from hardware in iteration {i+1}")


            self.save_data()
            logger.info(f"Iteration {i+1}/{iterations} completed")
            time.sleep(interval)

        logger.info("FIBERHOUND simulation completed")

def main():
    # Initialize and run FIBERHOUND system
    fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
    fiberhound.run(iterations=5, interval=1.0)

if __name__ == "__main__":
    main()

## Refactor and optimize

### Subtask:
Review the entire codebase, refactor where necessary, and add optimizations for a production system.


**Reasoning**:
Review the code for best practices, identify optimization areas, implement minor refactoring, add comments, and ensure basic error handling.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(1) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # 1. Read data from the established hardware connection stream.
            #    Example: `raw_data = self._hardware_connection.read_data()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols).
            #    Example: `parsed_data = parse_sensor_protocol(raw_data)`
            # 3. Manage buffering of incoming data.
            #    Example: Use a buffer to accumulate complete data packets.
            # 4. Apply sampling rates or data filtering as needed.
            #    Example: `filtered_data = apply_filters(parsed_data)`
            # 5. Implement error handling for data corruption, connection loss, or hardware read errors.
            #    Example: Check checksums, monitor connection status, handle exceptions during parsing.
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. The format should match
            #    the expected input of `process_sensor_data`:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool}

            # Simulate reading dummy sensor data for now
            dummy_sensor_data = []
            # Simulate variable sensor input, with a minimum of 1 detection to avoid empty reads
            num_simulated_detections = np.random.randint(1, 15)
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 dummy_sensor_data.append({
                     'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                     'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                     'timestamp': time.time(),
                     'is_active': np.random.choice([True, False], p=[0.8, 0.2])
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {e}, data: {data_point}")
                    continue

                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])

                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except (KeyError, TypeError, ValueError) as e:
                logger.warning(f"Skipping sensor data point due to data error: {e}, data: {data_point}")
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.
        Adds newly detected cables to the internal list and history.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        if not new_cables:
            logger.info("No new valid cables processed in this detection cycle.")
            return

        # Use a set for efficient checking of already detected cables based on start point
        # This assumes the start point uniquely identifies a cable segment for the purpose of duplicate detection.
        # A more robust approach might involve comparing both start and end points or using unique sensor IDs if available.
        detected_cable_tuples = {(c['start']) for c in self.detected_cables}

        added_count = 0
        for cable in new_cables:
            # Use tuple as it's hashable for set lookup
            cable_start_tuple = cable['start']
            if cable_start_tuple not in detected_cable_tuples:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                detected_cable_tuples.add(cable_start_tuple) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
        logger.info(f"Added {added_count} new cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        if added_count > 0:
             self.cable_history.sort(key=lambda x: x['timestamp'])

    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float] | None:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data (must include 'start' and 'end').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator.
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially close and most recently precedes the current one.
        # - Estimate the operator's likely position based on the connection point and direction.

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point (2D)

        # Find previous cables that are temporally before and spatially "connected" or close
        # Optimization: Iterate history backwards, stop after finding a connected cable or going back too far in time/count
        potential_previous_cables = [
            c for c in self.cable_history
            if c['timestamp'] < recent_cable['timestamp']
        ]

        best_previous_cable = None
        min_distance = float('inf')
        connection_threshold = 150.0 # Increased threshold slightly, can be tuned

        # Iterate through history in reverse order of time for recency
        for prev_cable in reversed(self.cable_history):
             if prev_cable['timestamp'] < recent_cable['timestamp']:
                  if 'start' in prev_cable and 'end' in prev_cable:
                    prev_end_2d = np.array(prev_cable['end'][:2])
                    curr_start_2d = np.array(recent_cable['start'][:2])
                    distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                    if distance < connection_threshold: # Found a potential connection
                         if distance < min_distance: # Found a better connection
                              min_distance = distance
                              best_previous_cable = prev_cable
                         # Optimization: If we find a connection within a very small distance,
                         # assume it's the correct previous segment and stop searching.
                         if distance < 10.0: # Example: within 10m
                              break # Found a very close previous cable, likely the one

                  # Optimization: Stop searching if we go back too many cables or too far in time
                  # This prevents iterating through the entire history every time.
                  time_diff = recent_cable['timestamp'] - prev_cable['timestamp']
                  if time_diff > 600: # Example: Stop if previous cable is older than 10 minutes
                      break


        if best_previous_cable:
            try:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                    # in the opposite direction of the estimated laying direction.
                    # This is a simplification; a real model would be more complex.
                    # Calculate average length only of cables used in tracing? Or overall history?
                    # Using overall length of recent history for simplicity and robustness.
                    recent_history_lengths = [c['length'] for c in self.cable_history[-50:] if 'length' in c] # Look at last 50 cables
                    avg_cable_length = np.mean(recent_history_lengths) if recent_history_lengths else 100.0
                    estimated_distance_behind_start = avg_cable_length * 0.75 # Adjusted heuristic distance
                    estimated_operator_pos_2d = curr_start_2d - normalized_direction * estimated_distance_behind_start

                    operator_location = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable (distance: {min_distance:.2f}m)")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
            except Exception as e:
                logger.error(f"Error during trajectory calculation: {e}. Defaulting to current start point.")
                operator_location = (recent_cable['start'][0], recent_cable['start'][1])

        else:
            logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             if 'start' in c and 'end' in c:
                points_2d.append(c['start'][:2])
                points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        def serialize_item(item):
             serializable_item = {}
             for k, v in item.items():
                  if isinstance(v, tuple):
                       serializable_item[k] = list(v) # Convert tuple to list
                  elif isinstance(v, np.ndarray):
                       serializable_item[k] = v.tolist() # Convert numpy array to list
                  elif isinstance(v, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                       serializable_item[k] = str(v) # Convert boolean to string
                  elif isinstance(v, (int, float, str, type(None))): # Include basic serializable types
                       serializable_item[k] = v
                  else:
                       # Fallback for potentially unhandled types - convert to string
                       logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' to string: {v}")
                       serializable_item[k] = str(v)
             return serializable_item

        # Apply serialization helper to lists of dictionaries/tuples
        detected_cables_serializable = [serialize_item(cable) for cable in self.detected_cables]
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Relies on `integrate_with_defense_stack` to provide mostly serializable data.
        Includes a robust check for remaining non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                else:
                    try:
                        # Check if it's already serializable by attempting a dump
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                # detect_cables returns the list of cables that were newly added
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active'
                             self.trace_operator(cable)
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predict_corridors
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

ERROR:__main__:Unexpected error processing sensor data point: cannot access local variable 'e' where it is not associated with a value, data: {'start': (-954.6832154170255, 10.43395445025567, 0), 'end': (-825.3330655078901, 600.1582511037307, 81.64089143433526), 'length': np.float64(609.238411871038), 'timestamp': 1749226010.3596354, 'is_active': np.True_}
ERROR:__main__:Unexpected error processing sensor data point: cannot access local variable 'e' where it is not associated with a value, data: {'start': (-619.5180473490313, -588.854677147614, 0), 'end': (896.224713980873, 824.400333470232, 59.160550613126006), 'length': np.float64(2073.2259438636447), 'timestamp': 1749226010.3600829, 'is_active': np.True_}
ERROR:__main__:Unexpected error processing sensor data point: cannot access local variable 'e' where it is not associated with a value, data: {'start': (-446.49889242018, 331.32865012678735, 0), 'end': (507.76560957605875, 625.0075657762407, 57.91592301887928), 'length': np.float64

**Reasoning**:
The previous command failed because tuple objects in `detected_cables` and `operator_locations` lists were not correctly converted to serializable types (like lists) before being included in the `defense_data` dictionary. The error message `'tuple' object has no attribute 'tolist'` confirms this. The `integrate_with_defense_stack` method needs to ensure these tuples are converted.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(1) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # 1. Read data from the established hardware connection stream.
            #    Example: `raw_data = self._hardware_connection.read_data()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols).
            #    Example: `parsed_data = parse_sensor_protocol(raw_data)`
            # 3. Manage buffering of incoming data.
            #    Example: Use a buffer to accumulate complete data packets.
            # 4. Apply sampling rates or data filtering as needed.
            # 5. Implement error handling for data corruption, connection loss, or hardware read errors.
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. The format should match
            #    the expected input of `process_sensor_data`:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool}

            # Simulate reading dummy sensor data for now
            dummy_sensor_data = []
            # Simulate variable sensor input, with a minimum of 1 detection to avoid empty reads
            num_simulated_detections = np.random.randint(1, 15)
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 dummy_sensor_data.append({
                     'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                     'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                     'timestamp': time.time(),
                     'is_active': np.random.choice([True, False], p=[0.8, 0.2])
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {data_point}")
                    continue

                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])

                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except (KeyError, TypeError, ValueError) as e:
                logger.warning(f"Skipping sensor data point due to data error: {e}, data: {data_point}")
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.
        Adds newly detected cables to the internal list and history.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        if not new_cables:
            logger.info("No new valid cables processed in this detection cycle.")
            return

        # Use a set for efficient checking of already detected cables based on start point
        # This assumes the start point uniquely identifies a cable segment for the purpose of duplicate detection.
        # A more robust approach might involve comparing both start and end points or using unique sensor IDs if available.
        detected_cable_tuples = {(c['start']) for c in self.detected_cables}

        added_count = 0
        for cable in new_cables:
            # Use tuple as it's hashable for set lookup
            cable_start_tuple = cable['start']
            if cable_start_tuple not in detected_cable_tuples:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                detected_cable_tuples.add(cable_start_tuple) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
        logger.info(f"Added {added_count} new cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis
        self.cable_history.sort(key=lambda x: x['timestamp'])

    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data.

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible.
        """
        if not recent_cable:
            logger.warning("Cannot trace operator: no recent cable provided.")
            return None

        logger.info(f"Tracing operator for recent cable: start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator.
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially close and most recently precedes the current one.
        # - Estimate the operator's likely position based on the connection point and direction.

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point

        # Find previous cables that are temporally before and spatially "connected" or close
        potential_previous_cables = [
            c for c in self.cable_history
            if c['timestamp'] < recent_cable['timestamp']
        ]

        if potential_previous_cables:
            # Sort by temporal proximity descending (most recent first)
            potential_previous_cables.sort(key=lambda x: x['timestamp'], reverse=True)

            best_previous_cable = None
            min_distance = float('inf')
            connection_threshold = 100.0 # Max distance between end of previous and start of current to consider connected

            # Find the closest previous cable end to the current cable start among recent cables
            curr_start_2d = np.array(recent_cable['start'][:2])
            # Consider only a limited number of the most recent previous cables for efficiency/relevance
            recent_previous_cables_subset = potential_previous_cables[:20] # Look at the last 20 potential previous cables

            for prev_cable in recent_previous_cables_subset:
                prev_end_2d = np.array(prev_cable['end'][:2])
                distance = np.linalg.norm(curr_start_2d - prev_end_2d)
                if distance < connection_threshold and distance < min_distance:
                    min_distance = distance
                    best_previous_cable = prev_cable

            if best_previous_cable:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                    # in the opposite direction of the estimated laying direction.
                    # This is a simplification; a real model would be more complex.
                    # Calculate average length only of cables used in tracing? Or overall history?
                    # Using overall history for simplicity here.
                    avg_cable_length = np.mean([c['length'] for c in self.cable_history]) if self.cable_history else 100.0
                    estimated_distance_behind_start = avg_cable_length * 0.5
                    estimated_operator_pos_2d = curr_start_2d - normalized_direction * estimated_distance_behind_start

                    operator_location = (estimated_operator_pos_2d[0], estimated_operator_pos_2d[1])
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable (timestamp: {best_previous_cable['timestamp']:.2f}, distance: {min_distance:.2f}m)")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
            else:
                logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")
        else:
            logger.info("No previous cable history found for trajectory analysis, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             points_2d.append(c['start'][:2])
             points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1)

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            # If you need labels per cable, you'd need to map them back (e.g., label for cable i is max(clusters[2*i], clusters[2*i+1]))
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error
            return np.full(len(points_2d_np), -1)


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            corridor_clusters = self.predict_corridors().tolist()
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Convert tuple elements within detected_cables to lists for JSON serialization
        # Also ensure boolean 'is_active' is handled here
        detected_cables_serializable = []
        for cable in self.detected_cables:
            serializable_cable = {}
            for k, v in cable.items():
                if isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert tuple to list
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert numpy array to list
                elif isinstance(v, bool):
                    serializable_cable[k] = str(v) # Convert boolean to string
                else:
                    serializable_cable[k] = v
            detected_cables_serializable.append(serializable_cable)

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) for loc in self.operator_locations]


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            serializable_data = self.integrate_with_defense_stack()

            # Recursive function to convert non-serializable types in the dictionary/list structure
            def make_serializable(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, bool):
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                # else:
                #     try:
                #         json.dumps(obj) # Check if it's already serializable
                #         return obj
                #     except TypeError:
                #         logger.warning(f"Found non-serializable type {type(obj)}: {obj}")
                #         return str(obj) # Fallback to string conversion

                return obj # Return as is if already serializable

            serializable_data_cleaned = make_serializable(serializable_data)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Detect cables from raw sensor data
                num_raw_data_points = len(raw_sensor_data) # Keep track of potential new cables from this read
                self.detect_cables(raw_sensor_data)

                # Step 4: Trace operator for newly added active cables
                # Iterate through the cables that were potentially added in this detection cycle.
                # This relies on detect_cables adding new items to the end of the list.
                # A more robust production system might use unique IDs or process cables based on their timestamp.
                # Need to be careful if process_sensor_data filters heavily, num_raw_data_points might not equal number of added cables.
                # A better approach is to get the list of newly added cables from detect_cables.
                # For now, let's process all cables returned by process_sensor_data if they are active.
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)
                for cable in processed_cables_this_cycle:
                     if cable.get('is_active', False): # Safely check 'is_active'
                         self.trace_operator(cable)


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 5: Predict corridors (using all detected cables)
            # Prediction happens implicitly when integrate_with_defense_stack is called
            # self.predict_corridors() # Can call explicitly if needed separately

            # Step 6: Integrate with defense stack and save data
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable


**Reasoning**:
The previous command failed because boolean values in the `detected_cables` list (specifically the `is_active` field) are not directly JSON serializable when included in the `defense_data` dictionary. Although the `save_data` method attempts to convert them to strings, the `integrate_with_defense_stack` method is called *before* this conversion happens, and the `integrate_with_defense_stack` method might return data that isn't fully serializable if called independently or if there are other boolean fields. The fix involves ensuring that the boolean values are converted to a serializable format (like strings) within the `integrate_with_defense_stack` method itself, or ensuring that the `save_data` method correctly handles boolean conversion for all relevant fields in the structured data it receives. The current error indicates the issue persists somewhere in the data preparation for JSON. The most reliable place to ensure JSON compatibility is just before serialization. I will modify `save_data` to iterate through the dictionary structure and explicitly convert boolean values to strings before calling `json.dump`.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(1) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # 1. Read data from the established hardware connection stream.
            #    Example: `raw_data = self._hardware_connection.read_data()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols).
            #    Example: `parsed_data = parse_sensor_protocol(raw_data)`
            # 3. Manage buffering of incoming data.
            #    Example: Use a buffer to accumulate complete data packets.
            # 4. Apply sampling rates or data filtering as needed.
            #    Example: `filtered_data = apply_filters(parsed_data)`
            # 5. Implement error handling for data corruption, connection loss, or hardware read errors.
            #    Example: Check checksums, monitor connection status, handle exceptions during parsing.
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. The format should match
            #    the expected input of `process_sensor_data`:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool}

            # Simulate reading dummy sensor data for now
            dummy_sensor_data = []
            # Simulate variable sensor input, with a minimum of 1 detection to avoid empty reads
            num_simulated_detections = np.random.randint(1, 15)
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 dummy_sensor_data.append({
                     'start_x': start_point[0], 'start_y': start_point[1], 'start_z': start_point[2],
                     'end_x': end_point[0], 'end_y': end_point[1], 'end_z': end_point[2],
                     'timestamp': time.time(),
                     'is_active': np.random.choice([True, False], p=[0.8, 0.2])
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, where each dictionary
                                      represents a potential cable detection from the sensor.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
        """
        processed_cables = []
        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {data_point}")
                    continue

                start_point = (data_point['start_x'], data_point['start_y'], data_point['start_z'])
                end_point = (data_point['end_x'], data_point['end_y'], data_point['end_z'])

                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': cable_length,
                        'timestamp': data_point.get('timestamp', time.time()), # Use provided timestamp or current time
                        'is_active': data_point.get('is_active', True) # Use provided status or assume active
                    })
            except (KeyError, TypeError, ValueError) as e:
                logger.warning(f"Skipping sensor data point due to data error: {e}, data: {data_point}")
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, sensor_input_data: List[Dict]) -> None:
        """
        Detect fibre optic cables in real-time using provided sensor data.
        Adds newly detected cables to the internal list and history.

        Args:
            sensor_input_data (List[Dict]): Raw sensor data input.
        """
        new_cables = self.process_sensor_data(sensor_input_data)
        if not new_cables:
            logger.info("No new valid cables processed in this detection cycle.")
            return

        # Use a set for efficient checking of already detected cables based on start point
        # This assumes the start point uniquely identifies a cable segment for the purpose of duplicate detection.
        # A more robust approach might involve comparing both start and end points or using unique sensor IDs if available.
        detected_cable_tuples = {(c['start']) for c in self.detected_cables}

        added_count = 0
        for cable in new_cables:
            # Use tuple as it's hashable for set lookup
            cable_start_tuple = cable['start']
            if cable_start_tuple not in detected_cable_tuples:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                detected_cable_tuples.add(cable_start_tuple) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
        logger.info(f"Added {added_count} new cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis
        self.cable_history.sort(key=lambda x: x['timestamp'])

    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data.

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible.
        """
        if not recent_cable:
            logger.warning("Cannot trace operator: no recent cable provided.")
            return None

        logger.info(f"Tracing operator for recent cable: start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator.
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially close and most recently precedes the current one.
        # - Estimate the operator's likely position based on the connection point and direction.

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point

        # Find previous cables that are temporally before and spatially "connected" or close
        potential_previous_cables = [
            c for c in self.cable_history
            if c['timestamp'] < recent_cable['timestamp']
        ]

        if potential_previous_cables:
            # Sort by temporal proximity descending (most recent first)
            potential_previous_cables.sort(key=lambda x: x['timestamp'], reverse=True)

            best_previous_cable = None
            min_distance = float('inf')
            connection_threshold = 100.0 # Max distance between end of previous and start of current to consider connected

            # Find the closest previous cable end to the current cable start among recent cables
            curr_start_2d = np.array(recent_cable['start'][:2])
            # Consider only a limited number of the most recent previous cables for efficiency/relevance
            recent_previous_cables_subset = potential_previous_cables[:20] # Look at the last 20 potential previous cables

            for prev_cable in recent_previous_cables_subset:
                prev_end_2d = np.array(prev_cable['end'][:2])
                distance = np.linalg.norm(curr_start_2d - prev_end_2d)
                if distance < connection_threshold and distance < min_distance:
                    min_distance = distance
                    best_previous_cable = prev_cable

            if best_previous_cable:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                    # in the opposite direction of the estimated laying direction.
                    # This is a simplification; a real model would be more complex.
                    # Calculate average length only of cables used in tracing? Or overall history?
                    # Using overall history for simplicity here.
                    avg_cable_length = np.mean([c['length'] for c in self.cable_history]) if self.cable_history else 100.0
                    estimated_distance_behind_start = avg_cable_length * 0.5
                    estimated_operator_pos_2d = curr_start_2d - normalized_direction * estimated_distance_behind_start

                    operator_location = (estimated_operator_pos_2d[0], estimated_operator_pos_2d[1])
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable (timestamp: {best_previous_cable['timestamp']:.2f}, distance: {min_distance:.2f}m)")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
            else:
                logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")
        else:
            logger.info("No previous cable history found for trajectory analysis, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             points_2d.append(c['start'][:2])
             points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1)

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            # If you need labels per cable, you'd need to map them back (e.g., label for cable i is max(clusters[2*i], clusters[2*i+1]))
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error
            return np.full(len(points_2d_np), -1)


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            corridor_clusters = self.predict_corridors().tolist()
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Convert tuple elements within detected_cables to lists for JSON serialization
        detected_cables_serializable = []
        for cable in self.detected_cables:
            serializable_cable = {}
            for k, v in cable.items():
                if isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert tuple to list
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert numpy array to list
                else:
                    serializable_cable[k] = v
            detected_cables_serializable.append(serializable_cable)

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) for loc in self.operator_locations]


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            serializable_data = self.integrate_with_defense_stack()

            # Recursive function to convert non-serializable types in the dictionary/list structure
            def make_serializable(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, bool):
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                # else:
                #     try:
                #         json.dumps(obj) # Check if it's already serializable
                #         return obj
                #     except TypeError:
                #         logger.warning(f"Found non-serializable type {type(obj)}: {obj}")
                #         return str(obj) # Fallback to string conversion

                return obj # Return as is if already serializable

            serializable_data_cleaned = make_serializable(serializable_data)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Detect cables from raw sensor data
                num_raw_data_points = len(raw_sensor_data) # Keep track of potential new cables from this read
                self.detect_cables(raw_sensor_data)

                # Step 4: Trace operator for newly added active cables
                # Iterate through the cables that were potentially added in this detection cycle.
                # This relies on detect_cables adding new items to the end of the list.
                # A more robust production system might use unique IDs or process cables based on their timestamp.
                # Need to be careful if process_sensor_data filters heavily, num_raw_data_points might not equal number of added cables.
                # A better approach is to get the list of newly added cables from detect_cables.
                # For now, let's process all cables detected in this cycle.
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)
                for cable in processed_cables_this_cycle:
                     # Check if this cable was actually added (not a duplicate) and is active
                     # This check is redundant if detect_cables only adds new ones and process_sensor_data filters active.
                     # Let's just process the cables returned by process_sensor_data if they are active.
                     if cable.get('is_active', False): # Safely check 'is_active'
                         self.trace_operator(cable)


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 5: Predict corridors (using all detected cables)
            # Prediction happens implicitly when integrate_with_defense_stack is called
            # self.predict_corridors() # Can call explicitly if needed separately

            # Step 6: Integrate with defense stack and save data
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable
ERROR:__main__:TypeError during data serialization for saving: Object of type bool is not JSON serializable


**Reasoning**:
The previous command still resulted in a `TypeError: Object of type bool is not JSON serializable`. This indicates that the recursive `make_serializable` function in `save_data` is not correctly handling boolean values everywhere they might appear in the nested dictionary structure returned by `integrate_with_defense_stack`. While the `system_status` was explicitly converted, booleans inside the `detected_cables` list (the `is_active` field) might still be causing issues. I will modify the `make_serializable` function to explicitly handle boolean types and convert them to strings. I will also review the `detected_cables_serializable` creation in `integrate_with_defense_stack` to ensure `is_active` is handled there as well, providing a more robust approach.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any
import logging
# Import libraries for potential real-time integration (placeholders for now)
# import requests # For HTTP POST/PUT
# import socket # For UDP
# import pika # For AMQP (RabbitMQ)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or self._hardware_connection is None: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # ... (previous comments remain relevant)
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. Ensure the format matches:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided ID)}


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
            Output format:
            {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
             'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []
        # Optimization: Pre-calculate numpy array for efficiency if many points
        # points_np = np.array([(dp['start_x'], dp['start_y'], dp['start_z'],
        #                        dp['end_x'], dp['end_y'], dp['end_z']) for dp in sensor_data])

        for data_point in sensor_data:
            try:
                # Validate required keys and data types
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {list(data_point.keys())}, expected: {required_keys}")
                    continue

                # Basic type validation
                if not all(isinstance(data_point.get(k), (int, float, np.number)) for k in ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp']):
                     logger.warning(f"Skipping sensor data point due to invalid numeric type: {data_point}")
                     continue
                if not isinstance(data_point.get('is_active'), (bool, np.bool_)):
                     logger.warning(f"Skipping sensor data point due to invalid boolean type for 'is_active': {data_point}")
                     continue
                if not isinstance(data_point.get('cable_id'), str):
                     logger.warning(f"Skipping sensor data point due to invalid string type for 'cable_id': {data_point}")
                     continue


                start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))

                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': float(data_point['timestamp']), # Ensure float
                        'is_active': bool(data_point['is_active']), # Ensure bool
                        'cable_id': str(data_point['cable_id']) # Ensure string
                    })
            except (KeyError, TypeError, ValueError) as e:
                logger.warning(f"Skipping sensor data point due to data error: {e}, data: {data_point}")
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict]) -> List[Dict]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            cable_id = cable.get('cable_id')
            if cable_id and cable_id not in detected_cable_ids:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            elif not cable_id:
                logger.warning(f"Skipping cable without a unique ID: {cable}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        if added_count > 0:
             self.cable_history.sort(key=lambda x: x['timestamp'])

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float] | None:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data (must include 'start' and 'end').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}, start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator.
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially close and most recently precedes the current one.
        # - Estimate the operator's likely position based on the connection point and direction.

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point (2D)

        # Find previous cables that are temporally before and spatially "connected" or close
        # Optimization: Iterate history backwards, stop after finding a connected cable or going back too far in time/count
        potential_previous_cables = [
            c for c in self.cable_history
            if c.get('timestamp', 0) < recent_cable.get('timestamp', float('inf')) # Safely check timestamp
        ]

        best_previous_cable = None
        min_distance = float('inf')
        connection_threshold = 150.0 # Increased threshold slightly, can be tuned

        # Iterate through history in reverse order of time for recency
        # Ensure history is sorted by timestamp before this loop if not already
        # self.cable_history.sort(key=lambda x: x.get('timestamp', 0)) # Already sorted in detect_cables

        for prev_cable in reversed(self.cable_history):
             # Safely check timestamp for comparison
             prev_timestamp = prev_cable.get('timestamp')
             recent_timestamp = recent_cable.get('timestamp')

             if prev_timestamp is not None and recent_timestamp is not None and prev_timestamp < recent_timestamp:
                  if 'start' in prev_cable and 'end' in prev_cable:
                    try:
                        prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                        curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
                        distance = np.linalg.norm(curr_start_2d - prev_end_2d) # Corrected: Compare end of previous to start of current

                        if distance < connection_threshold: # Found a potential connection
                             if distance < min_distance: # Found a better connection
                                  min_distance = distance
                                  best_previous_cable = prev_cable
                             # Optimization: If we find a connection within a very small distance,
                             # assume it's the correct previous segment and stop searching.
                             if distance < 10.0: # Example: within 10m
                                  break # Found a very close previous cable, likely the one
                    except (TypeError, ValueError) as e:
                         logger.warning(f"Error processing cable points for distance calculation: {e}. Skipping cable: {prev_cable}")
                         continue # Skip this previous cable if points are invalid

                  # Optimization: Stop searching if we go back too many cables or too far in time
                  # This prevents iterating through the entire history every time.
                  if recent_timestamp is not None and prev_timestamp is not None: # Ensure timestamps are valid for diff
                      time_diff = recent_timestamp - prev_timestamp
                      if time_diff > 600: # Example: Stop if previous cable is older than 10 minutes
                          break


        if best_previous_cable:
            try:
                prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float)
                curr_start_2d = np.array(recent_cable['start'][:2], dtype=float)
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                    # in the opposite direction of the estimated laying direction.
                    # This is a simplification; a real model would be more complex.
                    # Calculate average length only of cables used in tracing? Or overall history?
                    # Using overall length of recent history for simplicity and robustness.
                    recent_history_lengths = [c.get('length', 0) for c in self.cable_history[-50:]] # Look at last 50 cables, safely get length
                    avg_cable_length = np.mean(recent_history_lengths) if recent_history_lengths else 100.0
                    estimated_distance_behind_start = avg_cable_length * 0.75 # Adjusted heuristic distance
                    estimated_operator_pos_2d = curr_start_2d - normalized_direction * estimated_distance_behind_start

                    operator_location = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable ID {best_previous_cable.get('cable_id', 'N/A')} (distance: {min_distance:.2f}m)")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
                     operator_location = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Ensure floats
            except Exception as e:
                logger.error(f"Error during trajectory calculation: {e}. Defaulting to current start point.")
                operator_location = (float(recent_cable['start'][0]), float(recent_cable['start'][1])) # Ensure floats

        else:
            # If no connected previous cable found, use the start point of the current cable as a fallback
            operator_location = (float(recent_cable['start'][0]), float(recent_cable['start'][1])) # Ensure floats
            logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             if 'start' in c and 'end' in c:
                try:
                    points_2d.append(c['start'][:2])
                    points_2d.append(c['end'][:2])
                except (TypeError, ValueError) as e:
                    logger.warning(f"Skipping cable points for clustering due to data error: {e}, cable: {c}")


        points_2d_np = np.array(points_2d, dtype=float) # Ensure float dtype

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        def serialize_item(item):
             serializable_item = {}
             for k, v in item.items():
                  if isinstance(v, tuple):
                       serializable_item[k] = list(v) # Convert tuple to list
                  elif isinstance(v, np.ndarray):
                       serializable_item[k] = v.tolist() # Convert numpy array to list
                  elif isinstance(v, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                       serializable_item[k] = str(v) # Convert boolean to string
                  elif isinstance(v, (int, float, str, type(None))): # Include basic serializable types
                       serializable_item[k] = v
                  else:
                       # Fallback for potentially unhandled types - convert to string
                       logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' to string: {v}")
                       serializable_item[k] = str(v)
             return serializable_item

        # Apply serialization helper to lists of dictionaries/tuples
        detected_cables_serializable = [serialize_item(cable) for cable in self.detected_cables]
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.

        # Example using placeholder endpoints and libraries:
        # try:
        #     # Send cable detections via HTTP POST
        #     if defense_data['detected_cables']:
        #         api_url = defense_data['integration_endpoints']['cable_detection_api']
        #         logger.info(f"Attempting to send {len(defense_data['detected_cables'])} cable detections to {api_url} via HTTP POST...")
        #         # response = requests.post(api_url, json=defense_data['detected_cables'])
        #         # response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        #         # logger.info(f"Cable detections sent successfully. Status Code: {response.status_code}")
        # except Exception as e:
        #     logger.error(f"Failed to send cable detections to defense stack API: {e}")

        # try:
        #     # Send operator locations via UDP
        #     if defense_data['operator_locations']:
        #         udp_address = defense_data['integration_endpoints']['operator_location_protocol'].replace('UDP://', '')
        #         udp_ip, udp_port_str = udp_address.split(':')
        #         udp_port = int(udp_port_str)
        #         logger.info(f"Attempting to send {len(defense_data['operator_locations'])} operator locations to UDP {udp_ip}:{udp_port}...")
        #         # with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        #         #     message = json.dumps(defense_data['operator_locations']).encode('utf-8')
        #         #     s.sendto(message, (udp_ip, udp_port))
        #         # logger.info("Operator locations sent successfully via UDP.")
        # except Exception as e:
        #     logger.error(f"Failed to send operator locations via UDP: {e}")

        # try:
        #     # Publish corridor analysis results to a message queue (AMQP)
        #     if corridor_clusters:
        #         amqp_url = defense_data['integration_endpoints']['corridor_analysis_service'].replace('AMQP://', '')
        #         topic = amqp_url.split('/')[-1] # Extract topic from URL
        #         connection_params = pika.URLParameters(f'amqp://{amqp_url.replace(f"/{topic}", "")}')
        #         logger.info(f"Attempting to publish corridor analysis results to AMQP topic '{topic}'...")
        #         # connection = pika.BlockingConnection(connection_params)
        #         # channel = connection.channel()
        #         # # Declare a topic exchange (or use a default exchange)
        #         # channel.exchange_declare(exchange='fiberhound_exchange', exchange_type='topic')
        #         # message = json.dumps(corridor_clusters).encode('utf-8')
        #         # channel.basic_publish(exchange='fiberhound_exchange', routing_key=topic, body=message)
        #         # logger.info("Corridor analysis results published successfully to AMQP.")
        #         # connection.close()
        # except Exception as e:
        #     logger.error(f"Failed to publish corridor analysis results to AMQP: {e}")

        # try:
        #     # Send system health status via HTTP PUT
        #     status_url = defense_data['integration_endpoints']['system_health_status']
        #     logger.info(f"Attempting to send system health status to {status_url} via HTTP PUT...")
        #     # status_payload = {'status': defense_data['system_status'], 'timestamp': defense_data['timestamp']}
        #     # response = requests.put(status_url, json=status_payload)
        #     # response.raise_for_status()
        #     # logger.info(f"System health status sent successfully. Status Code: {response.status_code}")
        # except Exception as e:
        #     logger.error(f"Failed to send system health status via HTTP PUT: {e}")

        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Relies on `integrate_with_defense_stack` to provide mostly serializable data.
        Includes a robust check for remaining non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            # Ensure consistent indentation for all if/elif/else blocks
            def make_serializable_recursive(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple): # Explicitly handle tuple conversion
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                else:
                    try:
                        # Check if it's already serializable by attempting a dump
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.critical(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                # detect_cables returns the list of cables that were newly added
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active'
                             self.trace_operator(cable)
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predict_corridors implicitly
            self.integrate_with_defense_stack() # Generates data and logs intent and now includes send logic placeholders
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

**Reasoning**:
The STDERR output shows that the `process_sensor_data` method is skipping data points because they are missing required keys. The keys present in the skipped data points are 'start', 'end', 'length', 'timestamp', and 'is_active', which are the *output* format of `process_sensor_data`, not the expected *input* format. This means the simulated raw sensor data generated in `read_from_hardware` is still in the old output format rather than the expected input format for `process_sensor_data` (`start_x`, `start_y`, etc.). I need to fix `read_from_hardware` to simulate raw sensor data in the correct input format. Additionally, the `TypeError: Object of type bool is not JSON serializable` persists, suggesting the boolean to string conversion in `save_data` or `integrate_with_defense_stack` isn't catching all cases, or there's another boolean type (like `np.True_` or `np.False_`) that isn't being handled. I will refine the `make_serializable_robust` function to explicitly handle numpy boolean types (`np.bool_`) and also ensure `integrate_with_defense_stack`'s internal conversion handles numpy types correctly. Finally, I will continue the overall refactoring and optimization as planned.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # ... (previous comments remain relevant)
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. Ensure the format matches:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided ID)}


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': time.time(),
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
            Output format:
            {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
             'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []
        # Optimization: Pre-calculate numpy array for efficiency if many points
        # points_np = np.array([(dp['start_x'], dp['start_y'], dp['start_z'],
        #                        dp['end_x'], dp['end_y'], dp['end_z']) for dp in sensor_data])

        for data_point in sensor_data:
            try:
                # Validate required keys and data types
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {list(data_point.keys())}, expected: {required_keys}")
                    continue

                # Basic type validation
                if not all(isinstance(data_point.get(k), (int, float, np.number)) for k in ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp']):
                     logger.warning(f"Skipping sensor data point due to invalid numeric type: {data_point}")
                     continue
                if not isinstance(data_point.get('is_active'), (bool, np.bool_)):
                     logger.warning(f"Skipping sensor data point due to invalid boolean type for 'is_active': {data_point}")
                     continue
                if not isinstance(data_point.get('cable_id'), str):
                     logger.warning(f"Skipping sensor data point due to invalid string type for 'cable_id': {data_point}")
                     continue


                start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))

                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': float(data_point['timestamp']), # Ensure float
                        'is_active': bool(data_point['is_active']), # Ensure bool
                        'cable_id': str(data_point['cable_id']) # Ensure string
                    })
            except (KeyError, TypeError, ValueError) as e:
                logger.warning(f"Skipping sensor data point due to data error: {e}, data: {data_point}")
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict]) -> List[Dict]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            cable_id = cable.get('cable_id')
            if cable_id and cable_id not in detected_cable_ids:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            elif not cable_id:
                logger.warning(f"Skipping cable without a unique ID: {cable}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        if added_count > 0:
             self.cable_history.sort(key=lambda x: x['timestamp'])

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float] | None:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data (must include 'start' and 'end').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}, start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator.
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially close and most recently precedes the current one.
        # - Estimate the operator's likely position based on the connection point and direction.

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point (2D)

        # Find previous cables that are temporally before and spatially "connected" or close
        # Optimization: Iterate history backwards, stop after finding a connected cable or going back too far in time/count
        potential_previous_cables = [
            c for c in self.cable_history
            if c['timestamp'] < recent_cable['timestamp']
        ]

        best_previous_cable = None
        min_distance = float('inf')
        connection_threshold = 150.0 # Increased threshold slightly, can be tuned

        # Iterate through history in reverse order of time for recency
        for prev_cable in reversed(self.cable_history):
             if prev_cable['timestamp'] < recent_cable['timestamp']:
                  if 'start' in prev_cable and 'end' in prev_cable:
                    prev_end_2d = np.array(prev_cable['end'][:2])
                    curr_start_2d = np.array(recent_cable['start'][:2])
                    distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                    if distance < connection_threshold: # Found a potential connection
                         if distance < min_distance: # Found a better connection
                              min_distance = distance
                              best_previous_cable = prev_cable
                         # Optimization: If we find a connection within a very small distance,
                         # assume it's the correct previous segment and stop searching.
                         if distance < 10.0: # Example: within 10m
                              break # Found a very close previous cable, likely the one

                  # Optimization: Stop searching if we go back too many cables or too far in time
                  # This prevents iterating through the entire history every time.
                  time_diff = recent_cable['timestamp'] - prev_cable['timestamp']
                  if time_diff > 600: # Example: Stop if previous cable is older than 10 minutes
                      break


        if best_previous_cable:
            try:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                    # in the opposite direction of the estimated laying direction.
                    # This is a simplification; a real model would be more complex.
                    # Calculate average length only of cables used in tracing? Or overall history?
                    # Using overall length of recent history for simplicity and robustness.
                    recent_history_lengths = [c['length'] for c in self.cable_history[-50:] if 'length' in c] # Look at last 50 cables
                    avg_cable_length = np.mean(recent_history_lengths) if recent_history_lengths else 100.0
                    estimated_distance_behind_start = avg_cable_length * 0.75 # Adjusted heuristic distance
                    estimated_operator_pos_2d = curr_start_2d - normalized_direction * estimated_distance_behind_start

                    operator_location = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable ID {best_previous_cable.get('cable_id', 'N/A')} (distance: {min_distance:.2f}m)")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
            except Exception as e:
                logger.error(f"Error during trajectory calculation: {e}. Defaulting to current start point.")
                operator_location = (recent_cable['start'][0], recent_cable['start'][1])

        else:
            logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             if 'start' in c and 'end' in c:
                points_2d.append(c['start'][:2])
                points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        def serialize_item(item):
             serializable_item = {}
             for k, v in item.items():
                  if isinstance(v, tuple):
                       serializable_item[k] = list(v) # Convert tuple to list
                  elif isinstance(v, np.ndarray):
                       serializable_item[k] = v.tolist() # Convert numpy array to list
                  elif isinstance(v, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                       serializable_item[k] = str(v) # Convert boolean to string
                  elif isinstance(v, (int, float, str, type(None))): # Include basic serializable types
                       serializable_item[k] = v
                  else:
                       # Fallback for potentially unhandled types - convert to string
                       logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' to string: {v}")
                       serializable_item[k] = str(v)
             return serializable_item

        # Apply serialization helper to lists of dictionaries/tuples
        detected_cables_serializable = [serialize_item(cable) for cable in self.detected_cables]
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Relies on `integrate_with_defense_stack` to provide mostly serializable data.
        Includes a robust check for remaining non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                else:
                    try:
                        # Check if it's already serializable by attempting a dump
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                # detect_cables returns the list of cables that were newly added
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active'
                             self.trace_operator(cable)
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predict_corridors
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

## Summary:

### Data Analysis Key Findings

*   The initial script successfully replaced simulated sensor data with a method designed to accept real sensor data, including basic validation and filtering based on minimum cable length.
*   The `trace_operator` method was enhanced from a simple start-point assumption to a placeholder trajectory analysis that considers the temporal sequence and spatial relationship of detected cables to estimate the operator's location.
*   The script now includes outlined steps and placeholder methods (`initialize_hardware`, `read_from_hardware`) for integrating with real-time laser hardware, detailing necessary steps like interface identification, driver loading, connection establishment, configuration, and error handling.
*   The `integrate_with_defense_stack` method was expanded to include more specific placeholder API endpoints and communication protocols (HTTPS, UDP, AMQP) with comments indicating intended methods and data formats, providing a clearer outline for system integration.
*   Refactoring efforts addressed issues with JSON serialization of tuples and boolean types (including numpy booleans) and corrected a mismatch between simulated raw data format and the expected input format of the processing method.
*   The refactored code includes improved data validation, explicit type conversions for serialization, refined processing logic in `detect_cables` and `trace_operator`, enhanced logging, and basic error handling for file operations and data inconsistencies.
*   Minor optimizations, such as using sets for efficient duplicate checking and limiting the search space in trajectory analysis, were implemented.

### Insights or Next Steps

*   The placeholder trajectory analysis in `trace_operator` should be replaced with a more robust implementation using techniques like Kalman filters or sequence analysis for improved operator location estimation.
*   The hardware integration methods (`initialize_hardware`, `read_from_hardware`) need to be implemented with actual hardware communication libraries and protocols specific to the chosen laser/acoustic sensor.


# Task
Refactor the provided Python script for a cable detection system to replace simulated sensor data with real sensor inputs, enhance operator trajectory analysis using advanced techniques, integrate with real-time hardware (providing guidance for implementation), expand defense stack integration with placeholder API endpoints, and optimize the code for production.

## Replace simulated data

### Subtask:
Modify the `simulate_sensor_data` method to accept real sensor data as input.


**Reasoning**:
The previous attempt failed due to issues with JSON serialization of boolean and numpy types and a mismatch in the simulated data format. The `read_from_hardware` method was simulating data in the *output* format of `process_sensor_data` instead of the expected *input* format. The `save_data` method's recursive serialization also needed refinement to handle all boolean types explicitly. This code block addresses these issues by fixing the simulated data format in `read_from_hardware`, refining the `process_sensor_data` method's error handling and type conversions, and improving the recursive serialization in `save_data` to handle numpy booleans.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # ... (previous comments remain relevant)
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. Ensure the format matches:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided ID)}


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': time.time(),
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
            Output format:
            {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
             'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []
        # Optimization: Pre-calculate numpy array for efficiency if many points
        # points_np = np.array([(dp['start_x'], dp['start_y'], dp['start_z'],
        #                        dp['end_x'], dp['end_y'], dp['end_z']) for dp in sensor_data])

        for data_point in sensor_data:
            try:
                # Validate required keys and data types
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {list(data_point.keys())}, expected: {required_keys}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id'])

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue


                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict]) -> List[Dict]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            cable_id = cable.get('cable_id')
            if cable_id and cable_id not in detected_cable_ids:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            elif not cable_id:
                logger.warning(f"Skipping cable without a unique ID: {cable}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        if added_count > 0:
             self.cable_history.sort(key=lambda x: x['timestamp'])

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float] | None:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data (must include 'start' and 'end').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}, start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator.
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially close and most recently precedes the current one.
        # - Estimate the operator's likely position based on the connection point and direction.

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to start point (2D)

        # Find previous cables that are temporally before and spatially "connected" or close
        # Optimization: Iterate history backwards, stop after finding a connected cable or going back too far in time/count
        potential_previous_cables = [
            c for c in self.cable_history
            if c['timestamp'] < recent_cable['timestamp']
        ]

        best_previous_cable = None
        min_distance = float('inf')
        connection_threshold = 150.0 # Increased threshold slightly, can be tuned

        # Iterate through history in reverse order of time for recency
        for prev_cable in reversed(self.cable_history):
             if prev_cable['timestamp'] < recent_cable['timestamp']:
                  if 'start' in prev_cable and 'end' in prev_cable:
                    prev_end_2d = np.array(prev_cable['end'][:2])
                    curr_start_2d = np.array(recent_cable['start'][:2])
                    distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                    if distance < connection_threshold: # Found a potential connection
                         if distance < min_distance: # Found a better connection
                              min_distance = distance
                              best_previous_cable = prev_cable
                         # Optimization: If we find a connection within a very small distance,
                         # assume it's the correct previous segment and stop searching.
                         if distance < 10.0: # Example: within 10m
                              break # Found a very close previous cable, likely the one

                  # Optimization: Stop searching if we go back too many cables or too far in time
                  # This prevents iterating through the entire history every time.
                  time_diff = recent_cable['timestamp'] - prev_cable['timestamp']
                  if time_diff > 600: # Example: Stop if previous cable is older than 10 minutes
                      break


        if best_previous_cable:
            try:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance (e.g., half the average cable length)
                    # in the opposite direction of the estimated laying direction.
                    # This is a simplification; a real model would be more complex.
                    # Calculate average length only of cables used in tracing? Or overall history?
                    # Using overall length of recent history for simplicity and robustness.
                    recent_history_lengths = [c['length'] for c in self.cable_history[-50:] if 'length' in c] # Look at last 50 cables
                    avg_cable_length = np.mean(recent_history_lengths) if recent_history_lengths else 100.0
                    estimated_distance_behind_start = avg_cable_length * 0.75 # Adjusted heuristic distance
                    estimated_operator_pos_2d = curr_start_2d - normalized_direction * estimated_distance_behind_start

                    operator_location = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable ID {best_previous_cable.get('cable_id', 'N/A')} (distance: {min_distance:.2f}m)")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location, defaulting to current start point.")
            except Exception as e:
                logger.error(f"Error during trajectory calculation: {e}. Defaulting to current start point.")
                operator_location = (recent_cable['start'][0], recent_cable['start'][1])

        else:
            logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             if 'start' in c and 'end' in c:
                points_2d.append(c['start'][:2])
                points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        def serialize_item(item):
             serializable_item = {}
             for k, v in item.items():
                  if isinstance(v, tuple):
                       serializable_item[k] = list(v) # Convert tuple to list
                  elif isinstance(v, np.ndarray):
                       serializable_item[k] = v.tolist() # Convert numpy array to list
                  elif isinstance(v, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                       serializable_item[k] = str(v) # Convert boolean to string
                  elif isinstance(v, (int, float, str, type(None))): # Include basic serializable types
                       serializable_item[k] = v
                  else:
                       # Fallback for potentially unhandled types - convert to string
                       logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' to string: {v}")
                       serializable_item[k] = str(v)
             return serializable_item

        # Apply serialization helper to lists of dictionaries/tuples
        detected_cables_serializable = [serialize_item(cable) for cable in self.detected_cables]
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Relies on `integrate_with_defense_stack` to provide mostly serializable data.
        Includes a robust check for remaining non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                else:
                    try:
                        # Check if it's already serializable by attempting a dump
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                # detect_cables returns the list of cables that were newly added
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active'
                             self.trace_operator(cable)
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predict_corridors
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

## Enhance operator tracing

### Subtask:
Improve the `trace_operator` method to use more sophisticated trajectory analysis techniques.


**Reasoning**:
Modify the `trace_operator` method to implement a more sophisticated operator location estimation logic, moving away from the simple assumption that the operator is at the start point of the cable. This involves analyzing the sequence and properties of detected cables.



In [ ]:
import numpy as np
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # ... (previous comments remain relevant)
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. Ensure the format matches:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided ID)}


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': time.time(),
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
            Output format:
            {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
             'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []
        # Optimization: Pre-calculate numpy array for efficiency if many points
        # points_np = np.array([(dp['start_x'], dp['start_y'], dp['start_z'],
        #                        dp['end_x'], dp['end_y'], dp['end_z']) for dp in sensor_data])

        for data_point in sensor_data:
            try:
                # Validate required keys and data types
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {list(data_point.keys())}, expected: {required_keys}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id'])

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue


                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict]) -> List[Dict]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            cable_id = cable.get('cable_id')
            if cable_id and cable_id not in detected_cable_ids:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            elif not cable_id:
                logger.warning(f"Skipping cable without a unique ID: {cable}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        if added_count > 0:
             self.cable_history.sort(key=lambda x: x['timestamp'])

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float] | None:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data (must include 'start' and 'end').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}, start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Refined Placeholder Logic:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on average cable length.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to current cable start (2D)

        # Search for a spatially connected previous cable
        best_previous_cable = None
        min_distance = float('inf')
        # Tune this threshold based on expected operator movement speed and detection interval
        connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

        # Iterate through history in reverse order of time for recency
        # Limit the search to a reasonable number of recent cables for performance
        recent_history_subset = [c for c in self.cable_history if c['timestamp'] < recent_cable['timestamp']][-50:] # Look at up to the last 50 previous cables

        for prev_cable in reversed(recent_history_subset):
             if 'start' in prev_cable and 'end' in prev_cable:
                prev_end_2d = np.array(prev_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                if distance < connection_threshold: # Found a potential connection
                     if distance < min_distance: # Found a better connection (closer)
                          min_distance = distance
                          best_previous_cable = prev_cable
                     # Optimization: If we find a connection within a very small distance,
                     # assume it's the correct previous segment and stop searching.
                     if distance < 20.0: # Example: within 20m is a strong connection
                          break # Found a very close previous cable, likely the direct predecessor


        if best_previous_cable:
            try:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance based on the estimated laying speed
                    # (inferred from the distance and time between connected segments) and the time since the last detection.
                    # This is a simplified approach. A real system might use average speed or a dynamic estimate.

                    time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']
                    # Avoid division by zero or very small time differences
                    estimated_speed = np.linalg.norm(direction_vector) / time_diff if time_diff > 0.1 else 0.0

                    # Estimate position based on speed and time since last detection
                    # Assuming constant speed since the last detection (simplification)
                    # Operator is estimated to be 'estimated_speed * time_since_last_detection' distance back
                    # from the start of the current cable, along the reverse direction.
                    time_since_last_detection = time.time() - recent_cable['timestamp'] # Time from cable detection to now

                    # Re-calculate direction from start to end of the current cable as a potential laying direction
                    current_cable_direction = np.array(recent_cable['end'][:2]) - curr_start_2d
                    current_cable_norm = np.linalg.norm(current_cable_direction)
                    current_cable_normalized_direction = current_cable_direction / current_cable_norm if current_cable_norm > 0 else np.array([0.0, 0.0])

                    # Use the direction from previous end to current start for extrapolation
                    # Or use the direction of the current cable itself? Let's use the direction of the current cable
                    # as it's the most recent observation of movement.
                    # Extrapolate *backward* from the start of the current cable.
                    extrapolation_distance = estimated_speed * time_since_last_detection
                    # Limit extrapolation distance to avoid wild estimates
                    max_extrapolation_distance = 500.0 # Example limit
                    extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)


                    estimated_operator_pos_2d = curr_start_2d - current_cable_normalized_direction * extrapolation_distance


                    operator_location = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m.")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location or direction vector is zero, defaulting to current start point.")
            except Exception as e:
                logger.error(f"Error during trajectory calculation: {e}. Defaulting to current start point.")
                operator_location = (recent_cable['start'][0], recent_cable['start'][1])

        else:
            # Fallback: If no connected previous cable found, use the start point of the current cable
            # This is the same as the original simple logic.
            operator_location = (recent_cable['start'][0], recent_cable['start'][1])
            logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             if 'start' in c and 'end' in c:
                points_2d.append(c['start'][:2])
                points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        def serialize_item(item):
             serializable_item = {}
             for k, v in item.items():
                  if isinstance(v, tuple):
                       serializable_item[k] = list(v) # Convert tuple to list
                  elif isinstance(v, np.ndarray):
                       serializable_item[k] = v.tolist() # Convert numpy array to list
                  elif isinstance(v, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                       serializable_item[k] = str(v) # Convert boolean to string
                  elif isinstance(v, (int, float, str, type(None))): # Include basic serializable types
                       serializable_item[k] = v
                  else:
                       # Fallback for potentially unhandled types - convert to string
                       logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' to string: {v}")
                       serializable_item[k] = str(v)
             return serializable_item

        # Apply serialization helper to lists of dictionaries/tuples
        detected_cables_serializable = [serialize_item(cable) for cable in self.detected_cables]
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Relies on `integrate_with_defense_stack` to provide mostly serializable data.
        Includes a robust check for remaining non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                else:
                    try:
                        # Check if it's already serializable by attempting a dump
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                # detect_cables returns the list of cables that were newly added
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active'
                             self.trace_operator(cable)
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predict_corridors
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

## Integrate with hardware

### Subtask:
Outline the steps required to integrate the system with real-time hardware for laser-based cable detection.


## Expand defense stack integration

### Subtask:
Modify the `integrate_with_defense_stack` method to include actual API endpoints or communication protocols for sending detection data to a defense stack.


**Reasoning**:
Modify the `integrate_with_defense_stack` method to include more specific placeholder API endpoints and communication protocols with comments explaining their intended use, and ensure the data structure includes these.



In [ ]:
import numpy as np
import time
import json
from typing import List, Tuple, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ordered by timestamp.
        self.cable_history: List[Dict] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # Placeholder: Outline steps for real hardware integration
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Handle potential connection errors or hardware failures.
            #    Example: Check connection status, implement retries.

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}")
            self.hardware_initialized = False
            return False

    def read_from_hardware(self) -> List[Dict]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized:
            logger.warning("Hardware not initialized. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # Placeholder: Outline steps for reading real hardware data
            # ... (previous comments remain relevant)
            # 6. Convert raw data into a structured format (e.g., list of dictionaries)
            #    that can be processed by `process_sensor_data`. Ensure the format matches:
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided ID)}


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            for _ in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': time.time(),
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}")
            return []


    def process_sensor_data(self, sensor_data: List[Dict]) -> List[Dict]:
        """
        Process real laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List of dictionaries containing processed cable coordinates and metadata.
            Output format:
            {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
             'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []
        # Optimization: Pre-calculate numpy array for efficiency if many points
        # points_np = np.array([(dp['start_x'], dp['start_y'], dp['start_z'],
        #                        dp['end_x'], dp['end_y'], dp['end_z']) for dp in sensor_data])

        for data_point in sensor_data:
            try:
                # Validate required keys and data types
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    logger.warning(f"Skipping sensor data point due to missing required keys: {list(data_point.keys())}, expected: {required_keys}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id'])

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue


                # Calculate cable length using numpy for efficiency
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}")


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict]) -> List[Dict]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            cable_id = cable.get('cable_id')
            if cable_id and cable_id not in detected_cable_ids:
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.info(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            elif not cable_id:
                logger.warning(f"Skipping cable without a unique ID: {cable}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        if added_count > 0:
             self.cable_history.sort(key=lambda x: x['timestamp'])

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict) -> Tuple[float, float] | None:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently detected cable data (must include 'start' and 'end').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}, start={recent_cable['start']}, end={recent_cable['end']}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location = (recent_cable['start'][0], recent_cable['start'][1]) # Default to current cable start (2D)

        # Search for a spatially connected previous cable
        best_previous_cable = None
        min_distance = float('inf')
        # Tune this threshold based on expected operator movement speed and detection interval
        connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

        # Iterate through history in reverse order of time for recency
        # Limit the search to a reasonable number of recent cables for performance
        recent_history_subset = [c for c in self.cable_history if c['timestamp'] < recent_cable['timestamp']][-50:] # Look at up to the last 50 previous cables

        for prev_cable in reversed(recent_history_subset):
             if 'start' in prev_cable and 'end' in prev_cable:
                prev_end_2d = np.array(prev_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                if distance < connection_threshold: # Found a potential connection
                     if distance < min_distance: # Found a better connection (closer)
                          min_distance = distance
                          best_previous_cable = prev_cable
                     # Optimization: If we find a connection within a very small distance,
                     # assume it's the correct previous segment and stop searching.
                     if distance < 20.0: # Example: within 20m is a strong connection
                          break # Found a very close previous cable, likely the direct predecessor


        if best_previous_cable:
            try:
                prev_end_2d = np.array(best_previous_cable['end'][:2])
                curr_start_2d = np.array(recent_cable['start'][:2])
                # Calculate the direction vector from the end of the previous cable to the start of the current one
                direction_vector = curr_start_2d - prev_end_2d

                # Normalize the direction vector
                norm = np.linalg.norm(direction_vector)
                if norm > 0:
                    normalized_direction = direction_vector / norm

                    # Estimate operator location by extending backward from the current start point
                    # A simple heuristic: move back a certain distance based on the estimated laying speed
                    # (inferred from the distance and time between connected segments) and the time since the last detection.
                    # This is a simplified approach. A real system might use average speed or a dynamic estimate.

                    time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']
                    # Avoid division by zero or very small time differences
                    estimated_speed = np.linalg.norm(direction_vector) / time_diff if time_diff > 0.1 else 0.0

                    # Estimate position based on speed and time since last detection
                    # Assuming constant speed since the last detection (simplification)
                    # Operator is estimated to be 'estimated_speed * time_since_last_detection' distance back
                    # from the start of the current cable, along the reverse direction.
                    time_since_last_detection = time.time() - recent_cable['timestamp'] # Time from cable detection to now

                    # Re-calculate direction from start to end of the current cable as a potential laying direction
                    current_cable_direction = np.array(recent_cable['end'][:2]) - curr_start_2d
                    current_cable_norm = np.linalg.norm(current_cable_direction)
                    current_cable_normalized_direction = current_cable_direction / current_cable_norm if current_cable_norm > 0 else np.array([0.0, 0.0])

                    # Use the direction from previous end to current start for extrapolation
                    # Or use the direction of the current cable itself? Let's use the direction of the current cable
                    # as it's the most recent observation of movement.
                    # Extrapolate *backward* from the start of the current cable.
                    extrapolation_distance = estimated_speed * time_since_last_detection
                    # Limit extrapolation distance to avoid wild estimates
                    max_extrapolation_distance = 500.0 # Example limit
                    extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)


                    estimated_operator_pos_2d = curr_start_2d - current_cable_normalized_direction * extrapolation_distance


                    operator_location = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats
                    logger.info(f"Estimated operator location based on trajectory analysis with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m.")
                else:
                     logger.warning("Previous cable end and current cable start are at the same location or direction vector is zero, defaulting to current start point.")
            except Exception as e:
                logger.error(f"Error during trajectory calculation: {e}. Defaulting to current start point.")
                operator_location = (recent_cable['start'][0], recent_cable['start'][1])

        else:
            # Fallback: If no connected previous cable found, use the start point of the current cable
            # This is the same as the original simple logic.
            operator_location = (recent_cable['start'][0], recent_cable['start'][1])
            logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")


        self.operator_locations.append(operator_location)
        logger.info(f"Estimated operator location: {operator_location}")
        return operator_location

    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        for c in self.detected_cables:
             if 'start' in c and 'end' in c:
                points_2d.append(c['start'][:2])
                points_2d.append(c['end'][:2])

        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points
        if len(points_2d_np) < self.corridor_model.min_samples:
            logger.warning(f"Insufficient data ({len(points_2d_np)} points) for corridor clustering (requires at least {self.corridor_model.min_samples}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN.")

            # Return the cluster labels for each point (start and end points interleaved)
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}")
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}")
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        def serialize_item(item):
             serializable_item = {}
             for k, v in item.items():
                  if isinstance(v, tuple):
                       serializable_item[k] = list(v) # Convert tuple to list
                  elif isinstance(v, np.ndarray):
                       serializable_item[k] = v.tolist() # Convert numpy array to list
                  elif isinstance(v, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                       serializable_item[k] = str(v) # Convert boolean to string
                  elif isinstance(v, (int, float, str, type(None))): # Include basic serializable types
                       serializable_item[k] = v
                  else:
                       # Fallback for potentially unhandled types - convert to string
                       logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' to string: {v}")
                       serializable_item[k] = str(v)
             return serializable_item

        # Apply serialization helper to lists of dictionaries/tuples
        detected_cables_serializable = [serialize_item(cable) for cable in self.detected_cables]
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data = {
            'timestamp': time.time(),
            'detected_cables': detected_cables_serializable,
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload)
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections', # Method: POST
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON)
                'operator_location_protocol': 'UDP://192.168.1.100:5005', # Protocol: UDP
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf)
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis', # Protocol: AMQP
                # Secure HTTP endpoint for system health and status (e.g., JSON payload)
                'system_health_status': 'https://defense.system.com/api/v1/status' # Method: PUT
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data.")
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # Example: send_data_to_api(defense_data['cable_detection_api'], defense_data['detected_cables'])
        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Relies on `integrate_with_defense_stack` to provide mostly serializable data.
        Includes a robust check for remaining non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should handle main types now.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj):
                if isinstance(obj, dict):
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    return list(obj) # Convert tuples to lists
                elif isinstance(obj, np.ndarray):
                    return obj.tolist() # Convert numpy arrays to lists
                elif isinstance(obj, (bool, np.bool_)): # Explicitly handle bool and numpy bool
                    return str(obj) # Convert booleans to strings
                # Add other types as needed (e.g., datetime objects)
                else:
                    try:
                        # Check if it's already serializable by attempting a dump
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                json.dump(serializable_data_cleaned, f, indent=2)
            logger.info(f"Saved detection data to {filename}")
        except TypeError as e:
            logger.error(f"TypeError during data serialization for saving: {e}")
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}")
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}")


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                # detect_cables returns the list of cables that were newly added
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active'
                             self.trace_operator(cable)
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predict_corridors
            self.integrate_with_defense_stack() # Generates data and logs intent
            self.save_data()

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed: {e}")
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}")


if __name__ == "__main__":
    main()

## Refactor and optimize

### Subtask:
Review the entire codebase, refactor where necessary, and add optimizations for a production system.


**Reasoning**:
Review the code for best practices, identify optimization areas, implement minor refactoring, add comments, and ensure basic error handling. This includes refining error handling in `process_sensor_data` and `save_data`, clarifying comments, adding type hints where missing, and ensuring the `make_serializable_recursive` function correctly handles numpy types and bools.



In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any, Optional # Added Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.

    Attributes:
        detection_range (float): Maximum detection range in meters.
        min_cable_length (float): Minimum cable length to consider for detection.
        detected_cables (List[Dict]): Stores all detected cables over time.
        operator_locations (List[Tuple[float, float]]): Stores estimated operator locations (2D).
        corridor_model (DBSCAN): Clustering model for corridor detection.
        cable_history (List[Dict]): Stores historical cable data for trajectory analysis.
        hardware_initialized (bool): Flag to track hardware initialization status.
        _cable_counter (int): Simple counter for assigning unique IDs to simulated cables.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.

        Raises:
            ValueError: If detection_range or min_cable_length are not positive.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict[str, Any]] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned for density.
        # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        # min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ideally ordered by timestamp.
        self.cable_history: List[Dict[str, Any]] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        This method should contain the actual logic for connecting to and configuring
        the sensor hardware. It's currently a placeholder simulation.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Initialization ---
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Store the connection object if needed for subsequent reads.
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Implement robust error handling and retry mechanisms for connection or configuration failures.
            # --- End Placeholder ---

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}", exc_info=True) # Log exception details
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict[str, Any]]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        This method should contain the actual logic for reading data from the
        initialized sensor hardware. It's currently a placeholder simulation.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format for each dictionary:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or not self._hardware_connection: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Reading ---
            # 1. Read data from the established hardware connection stream (`self._hardware_connection`).
            #    Example: `raw_sensor_output = self._hardware_connection.read_stream()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols). Parse the raw output.
            #    Example: `parsed_output = parse_sensor_protocol(raw_sensor_output)`
            # 3. Manage buffering of incoming data to ensure complete data packets are processed.
            # 4. Apply sampling rates or data filtering as needed before formatting.
            # 5. Implement robust error handling for data corruption, connection loss during read, or hardware read errors.
            # 6. Convert parsed raw data into the structured format (list of dictionaries)
            #    expected by `process_sensor_data`. Ensure correct data types (float, bool, str) and required keys.
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided unique ID)}
            # --- End Placeholder ---


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT for process_sensor_data
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}", exc_info=True) # Log exception details
            # Consider marking hardware as uninitialized if a critical read error occurs
            # self.hardware_initialized = False
            return []


    def process_sensor_data(self, sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process raw laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List[Dict]: List of dictionaries containing processed cable coordinates and metadata.
                        Output format:
                        {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
                         'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []

        for data_point in sensor_data:
            try:
                # Validate required keys and data types
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    missing_keys = [key for key in required_keys if key not in data_point]
                    logger.warning(f"Skipping sensor data point due to missing required keys: {missing_keys}, data: {data_point}")
                    continue

                # Basic type validation and conversion - use a helper function for clarity
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id']) # Ensure cable_id is a string

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue
                except Exception as e:
                     logger.error(f"Unexpected error during type conversion for sensor data point: {e}, data: {data_point}", exc_info=True)
                     continue # Skip this data point on unexpected conversion error


                # Calculate cable length using numpy for efficiency
                # Ensure points are numpy arrays for calculation
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                # Catch any other unexpected errors during processing a single data point
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}", exc_info=True)


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history based on unique ID.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            # Validate presence of cable_id before attempting to add
            cable_id = cable.get('cable_id')
            if cable_id is None:
                 logger.warning(f"Skipping cable detection due to missing 'cable_id': {cable}")
                 continue # Skip this cable if ID is missing

            if cable_id not in detected_cable_ids:
                # Add to detected_cables and history
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.debug(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        # Sorting the entire history repeatedly might become inefficient for very long runs.
        # Consider using a structure that maintains order or sorting a smaller subset if needed.
        if added_count > 0:
             # Ensure 'timestamp' key exists before sorting
             if all('timestamp' in c for c in self.cable_history):
                 self.cable_history.sort(key=lambda x: x['timestamp'])
             else:
                 logger.warning("Skipping history sort: 'timestamp' key missing in some history entries.")

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict[str, Any]) -> Optional[Tuple[float, float]]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently processed cable data
                                 (must include 'start', 'end', 'timestamp').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        # Validate input
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable or 'timestamp' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location_2d: Optional[Tuple[float, float]] = None # Initialize as None

        try:
            curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
            curr_end_2d = np.array(recent_cable['end'][:2], dtype=float) # Ensure float dtype

            # Search for a spatially connected previous cable among recent history
            best_previous_cable = None
            min_distance = float('inf')
            # Tune this threshold based on expected operator movement speed and detection interval
            connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

            # Iterate through history in reverse order of time for recency
            # Limit the search to a reasonable number of recent cables for performance
            # Filter history for valid cables with required keys and timestamp before recent cable
            valid_previous_cables = [
                 c for c in self.cable_history
                 if c.get('timestamp') is not None and c['timestamp'] < recent_cable['timestamp']
                 and 'start' in c and 'end' in c # Ensure start/end are present
            ]
            recent_previous_cables_subset = valid_previous_cables[-50:] # Look at up to the last 50 valid previous cables

            for prev_cable in reversed(recent_previous_cables_subset):
                 prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                 distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                 if distance < connection_threshold: # Found a potential connection
                      if distance < min_distance: # Found a better connection (closer)
                           min_distance = distance
                           best_previous_cable = prev_cable
                      # Optimization: If we find a connection within a very small distance,
                      # assume it's the correct previous segment and stop searching.
                      if distance < 20.0: # Example: within 20m is a strong connection
                           break # Found a very close previous cable, likely the direct predecessor


            if best_previous_cable:
                 # Estimate operator location based on the connected previous cable
                 prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float) # Ensure float dtype
                 time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']

                 if time_diff > 0.1: # Avoid division by zero or near-zero time differences
                     # Use the direction vector from the end of the previous cable to the start of the current one
                     # as the likely laying direction segment.
                     laying_direction_vector = curr_start_2d - prev_end_2d
                     vector_norm = np.linalg.norm(laying_direction_vector)

                     if vector_norm > 0:
                         estimated_speed = vector_norm / time_diff

                         # Estimate position based on speed and time since the *recent cable* was detected
                         # This assumes the operator continued laying at the estimated speed since the last detection.
                         time_since_recent_detection = time.time() - recent_cable['timestamp'] # Time from cable detection to now
                         extrapolation_distance = estimated_speed * time_since_recent_detection

                         # Limit extrapolation distance to avoid unrealistic jumps
                         max_extrapolation_distance = 500.0 # Example limit, tune based on expected speeds
                         extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)

                         # The operator is estimated to be *behind* the start of the recent cable,
                         # along the *reverse* direction of the laying vector.
                         if vector_norm > 0: # Normalize only if norm is positive
                             normalized_reverse_direction = -laying_direction_vector / vector_norm
                             estimated_operator_pos_2d = curr_start_2d + normalized_reverse_direction * extrapolation_distance
                             operator_location_2d = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats

                             logger.info(f"Estimated operator location based on trajectory with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m (connection distance: {min_distance:.2f}m).")
                         else:
                              logger.warning("Calculated laying direction vector has zero norm. Defaulting to current start point.")
                              operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                     else:
                          logger.warning("Time difference between connected cables is too small (<= 0.1s) or vector norm is zero. Cannot estimate speed/extrapolate. Defaulting to current start point.")
                          operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                else:
                     # Fallback if time_diff is not positive
                     logger.warning("Time difference between connected cables is not positive. Defaulting to current start point.")
                     operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))

            else:
                 # Fallback: If no spatially connected previous cable found, use the start point of the current cable
                 operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Default to current cable start (2D)
                 logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

            # Ensure operator_location_2d is set even in fallbacks
            if operator_location_2d is not None:
                 self.operator_locations.append(operator_location_2d)
                 logger.info(f"Estimated operator location: {operator_location_2d}")
                 return operator_location_2d
            else:
                 # This case should ideally not be reached with the fallbacks, but included for safety.
                 logger.error("Operator location estimation failed unexpectedly.")
                 return None

        except Exception as e:
            logger.error(f"An error occurred during operator trajectory tracing: {e}", exc_info=True)
            # In case of error, return None and do not append to operator_locations to avoid invalid data
            return None


    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables in 2D.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data or on error.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        valid_cables_count = 0
        for c in self.detected_cables:
             if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
                try:
                    # Ensure points are convertible to float
                    start_point_2d = (float(c['start'][0]), float(c['start'][1]))
                    end_point_2d = (float(c['end'][0]), float(c['end'][1]))
                    points_2d.append(start_point_2d)
                    points_2d.append(end_point_2d)
                    valid_cables_count += 1
                except (ValueError, TypeError):
                    logger.warning(f"Skipping cable with invalid start/end coordinates for clustering: {c}")


        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points to form a cluster (excluding noise)
        # and at least 2 points in total to run fit_predict.
        if len(points_2d_np) < max(2, self.corridor_model.min_samples):
            logger.warning(f"Insufficient valid data ({len(points_2d_np)} points from {valid_cables_count} cables) for corridor clustering (requires at least {max(2, self.corridor_model.min_samples)}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            # Optimization: Consider pre-scaling data if coordinates span very different ranges, though unlikely for this data.
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN from {len(points_2d_np)} points.")

            # Return the cluster labels for each point (start and end points interleaved)
            # Mapping labels back to cables (e.g., for per-cable corridor assignment)
            # would require associating the original cable index with the points_2d list creation.
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}", exc_info=True)
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}", exc_info=True)
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        # This function is designed to handle the specific structure of cable dictionaries.
        def serialize_cable_item(cable_dict: Dict[str, Any]) -> Dict[str, Any]:
            serializable_cable = {}
            for k, v in cable_dict.items():
                if k in ['start', 'end'] and isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert point tuples to lists
                elif k == 'length' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float/int
                elif k == 'timestamp' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float
                elif k == 'is_active' and isinstance(v, (bool, np.bool_)):
                     serializable_cable[k] = str(v) # Convert bool/numpy bool to string
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert other numpy arrays to lists
                elif isinstance(v, (int, float, str, type(None))): # Already serializable types
                    serializable_cable[k] = v
                else:
                    # Fallback for potentially unhandled types - convert to string
                    logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' in cable data to string: {v}")
                    serializable_cable[k] = str(v)
            return serializable_cable

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data: Dict[str, Any] = {
            'timestamp': time.time(),
            'detected_cables': [serialize_cable_item(cable) for cable in self.detected_cables], # Use helper for cables
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload).
                # Method: POST
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON).
                # Protocol: UDP
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf).
                # Protocol: AMQP
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis',
                # Secure HTTP endpoint for system health and status (e.g., JSON payload).
                # Method: PUT
                'system_health_status': 'https://defense.system.com/api/v1/status'
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data structure.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.
        # Example:
        # try:
        #     send_to_api(defense_data['integration_endpoints']['cable_detection_api'], defense_data['detected_cables'])
        #     send_to_udp(defense_data['integration_endpoints']['operator_location_protocol'], defense_data['operator_locations'])
        #     publish_message(defense_data['integration_endpoints']['corridor_analysis_service'], defense_data['corridor_clusters'])
        # except Exception as e:
        #     logger.error(f"Failed to send data to defense stack: {e}")
        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should provide mostly serializable data.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj: Any) -> Any:
                if isinstance(obj, dict):
                    # Recursively process dictionary values
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    # Recursively process list elements
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    # Convert tuples to lists (common JSON conversion)
                    return list(obj)
                elif isinstance(obj, np.ndarray):
                    # Convert numpy arrays to lists
                    return obj.tolist()
                elif isinstance(obj, (bool, np.bool_)):
                    # Explicitly convert boolean types (Python bool and numpy bool) to string
                    return str(obj)
                elif isinstance(obj, np.number):
                    # Convert other numpy numbers (int, float) to standard Python types
                    return obj.item() # .item() converts numpy scalar to Python scalar
                # Add other specific type conversions here if needed (e.g., datetime)
                # elif isinstance(obj, datetime.datetime):
                #     return obj.isoformat()
                elif isinstance(obj, (int, float, str, type(None))):
                    # These types are already JSON serializable, return as is
                    return obj
                else:
                    # Fallback for any remaining unhandled types: attempt serialization check
                    # and convert to string if not serializable.
                    try:
                        # Check if it's already serializable by attempting a dump (without logging recursion)
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            # Apply the recursive conversion to the data structure
            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            # Use a more robust JSON dump that handles edge cases or provides better error messages if needed
            # json.JSONEncoder().encode() or a custom encoder could be used, but simple dump is often sufficient
            # once basic types are handled.
            with open(filename, 'w') as f:
                # Use allow_nan=False for stricter JSON compliance if needed
                json.dump(serializable_data_cleaned, f, indent=2, allow_nan=True)
            logger.info(f"Saved detection data to {filename}")

        except TypeError as e:
            # This catch block should now be less frequent if make_serializable_recursive is robust
            logger.error(f"TypeError during data serialization for saving: {e}", exc_info=True)
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}", exc_info=True)
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}", exc_info=True)


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        # In a real system, initialize hardware only once or manage reconnection logic
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    logger.info(f"Tracing operator for {len(newly_added_cables)} newly added cables...")
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active', default to False
                             estimated_location = self.trace_operator(cable)
                             # In a real system, you might want to handle the case where estimated_location is None
                             # and log/report the failure to trace.
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predicting corridors implicitly
            defense_data_output = self.integrate_with_defense_stack() # Generates data and logs intent
            # Note: Actual sending to defense stack would happen INSIDE integrate_with_defense_stack
            # or be triggered immediately after calling it.

            self.save_data() # Save data to local file for record-keeping/debugging

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        # Example of potentially reading config from a file or env vars
        # detection_range = float(os.getenv('DETECTION_RANGE', 5000.0))
        # min_cable_length = float(os.getenv('MIN_CABLE_LENGTH', 100.0))
        # iterations = int(os.getenv('RUN_ITERATIONS', 5))
        # interval = float(os.getenv('RUN_INTERVAL', 1.0))

        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed due to invalid parameters: {e}", exc_info=True)
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}", exc_info=True)


if __name__ == "__main__":
    # When run as a script, execute main
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 394)

**Reasoning**:
The previous command failed due to an `IndentationError`. This indicates a syntax issue with inconsistent indentation, likely in the `save_data` method where the `make_serializable_recursive` function is defined or used, or possibly in the `trace_operator` method's complex logic. I need to fix the indentation errors in the code. I will regenerate the full code block with corrected indentation, paying close attention to the `save_data` and `trace_operator` methods.



In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any, Optional # Added Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.

    Attributes:
        detection_range (float): Maximum detection range in meters.
        min_cable_length (float): Minimum cable length to consider for detection.
        detected_cables (List[Dict]): Stores all detected cables over time.
        operator_locations (List[Tuple[float, float]]): Stores estimated operator locations (2D).
        corridor_model (DBSCAN): Clustering model for corridor detection.
        cable_history (List[Dict]): Stores historical cable data for trajectory analysis.
        hardware_initialized (bool): Flag to track hardware initialization status.
        _cable_counter (int): Simple counter for assigning unique IDs to simulated cables.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.

        Raises:
            ValueError: If detection_range or min_cable_length are not positive.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict[str, Any]] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned for density.
        # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        # min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ideally ordered by timestamp.
        self.cable_history: List[Dict[str, Any]] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        This method should contain the actual logic for connecting to and configuring
        the sensor hardware. It's currently a placeholder simulation.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Initialization ---
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Store the connection object if needed for subsequent reads.
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Implement robust error handling and retry mechanisms for connection or configuration failures.
            # --- End Placeholder ---

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}", exc_info=True) # Log exception details
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict[str, Any]]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        This method should contain the actual logic for reading data from the
        initialized sensor hardware. It's currently a placeholder simulation.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or self._hardware_connection is None: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Reading ---
            # 1. Read data from the established hardware connection stream (`self._hardware_connection`).
            #    Example: `raw_sensor_output = self._hardware_connection.read_stream()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols). Parse the raw output.
            #    Example: `parsed_output = parse_sensor_protocol(raw_sensor_output)`
            # 3. Manage buffering of incoming data to ensure complete data packets are processed.
            # 4. Apply sampling rates or data filtering as needed before formatting.
            # 5. Implement robust error handling for data corruption, connection loss during read, or hardware read errors.
            # 6. Convert parsed raw data into the structured format (list of dictionaries)
            #    expected by `process_sensor_data`. Ensure correct data types (float, bool, str) and required keys.
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided unique ID)}
            # --- End Placeholder ---


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT for process_sensor_data
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}", exc_info=True) # Log exception details
            # Consider marking hardware as uninitialized if a critical read error occurs
            # self.hardware_initialized = False
            return []


    def process_sensor_data(self, sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process raw laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List[Dict]: List of dictionaries containing processed cable coordinates and metadata.
                        Output format:
                        {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
                         'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []

        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    missing_keys = [key for key in required_keys if key not in data_point]
                    logger.warning(f"Skipping sensor data point due to missing required keys: {missing_keys}, data: {data_point}")
                    continue

                # Basic type validation and conversion - use a helper function for clarity
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id']) # Ensure cable_id is a string

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue
                except Exception as e:
                     logger.error(f"Unexpected error during type conversion for sensor data point: {e}, data: {data_point}", exc_info=True)
                     continue # Skip this data point on unexpected conversion error


                # Calculate cable length using numpy for efficiency
                # Ensure points are numpy arrays for calculation
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                # Catch any other unexpected errors during processing a single data point
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}", exc_info=True)


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history based on unique ID.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            # Validate presence of cable_id before attempting to add
            cable_id = cable.get('cable_id')
            if cable_id is None:
                 logger.warning(f"Skipping cable detection due to missing 'cable_id': {cable}")
                 continue # Skip this cable if ID is missing

            if cable_id not in detected_cable_ids:
                # Add to detected_cables and history
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.debug(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        # Sorting the entire history repeatedly might become inefficient for very long runs.
        # Consider using a structure that maintains order or sorting a smaller subset if needed.
        if added_count > 0:
             # Ensure 'timestamp' key exists before sorting
             if all('timestamp' in c for c in self.cable_history):
                 self.cable_history.sort(key=lambda x: x['timestamp'])
             else:
                 logger.warning("Skipping history sort: 'timestamp' key missing in some history entries.")

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict[str, Any]) -> Optional[Tuple[float, float]]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently processed cable data
                                 (must include 'start', 'end', 'timestamp').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        # Validate input
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable or 'timestamp' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location_2d: Optional[Tuple[float, float]] = None # Initialize as None

        try:
            curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
            curr_end_2d = np.array(recent_cable['end'][:2], dtype=float) # Ensure float dtype

            # Search for a spatially connected previous cable among recent history
            best_previous_cable = None
            min_distance = float('inf')
            # Tune this threshold based on expected operator movement speed and detection interval
            connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

            # Iterate through history in reverse order of time for recency
            # Limit the search to a reasonable number of recent cables for performance
            # Filter history for valid cables with required keys and timestamp before recent cable
            valid_previous_cables = [
                 c for c in self.cable_history
                 if c.get('timestamp') is not None and c['timestamp'] < recent_cable['timestamp']
                 and 'start' in c and 'end' in c # Ensure start/end are present
            ]
            recent_previous_cables_subset = valid_previous_cables[-50:] # Look at up to the last 50 valid previous cables

            for prev_cable in reversed(recent_previous_cables_subset):
                 prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                 distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                 if distance < connection_threshold: # Found a potential connection
                      if distance < min_distance: # Found a better connection (closer)
                           min_distance = distance
                           best_previous_cable = prev_cable
                      # Optimization: If we find a connection within a very small distance,
                      # assume it's the correct previous segment and stop searching.
                      if distance < 20.0: # Example: within 20m is a strong connection
                           break # Found a very close previous cable, likely the direct predecessor


            if best_previous_cable:
                 # Estimate operator location based on the connected previous cable
                 prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float) # Ensure float dtype
                 time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']

                 if time_diff > 0.1: # Avoid division by zero or very small time differences
                     # Use the direction vector from the end of the previous cable to the start of the current one
                     # as the likely laying direction segment.
                     laying_direction_vector = curr_start_2d - prev_end_2d
                     vector_norm = np.linalg.norm(laying_direction_vector)

                     if vector_norm > 0:
                         estimated_speed = vector_norm / time_diff

                         # Estimate position based on speed and time since the *recent cable* was detected
                         # This assumes the operator continued laying at the estimated speed since the last detection.
                         time_since_recent_detection = time.time() - recent_cable['timestamp'] # Time from cable detection to now
                         extrapolation_distance = estimated_speed * time_since_recent_detection

                         # Limit extrapolation distance to avoid unrealistic jumps
                         max_extrapolation_distance = 500.0 # Example limit, tune based on expected speeds
                         extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)

                         # The operator is estimated to be *behind* the start of the recent cable,
                         # along the *reverse* direction of the laying vector.
                         if vector_norm > 0: # Normalize only if norm is positive
                             normalized_reverse_direction = -laying_direction_vector / vector_norm
                             estimated_operator_pos_2d = curr_start_2d + normalized_reverse_direction * extrapolation_distance
                             operator_location_2d = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats

                             logger.info(f"Estimated operator location based on trajectory with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m (connection distance: {min_distance:.2f}m).")
                         else:
                              logger.warning("Calculated laying direction vector has zero norm. Defaulting to current start point.")
                              operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                     else:
                          logger.warning("Time difference between connected cables is too small (<= 0.1s) or vector norm is zero. Cannot estimate speed/extrapolate. Defaulting to current start point.")
                          operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                else:
                     # Fallback if time_diff is not positive
                     logger.warning("Time difference between connected cables is not positive. Defaulting to current start point.")
                     operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))

            else:
                 # Fallback: If no spatially connected previous cable found, use the start point of the current cable
                 operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Default to current cable start (2D)
                 logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

            # Ensure operator_location_2d is set even in fallbacks
            if operator_location_2d is not None:
                 self.operator_locations.append(operator_location_2d)
                 logger.info(f"Estimated operator location: {operator_location_2d}")
                 return operator_location_2d
            else:
                 # This case should ideally not be reached with the fallbacks, but included for safety.
                 logger.error("Operator location estimation failed unexpectedly.")
                 return None

        except Exception as e:
            logger.error(f"An error occurred during operator trajectory tracing: {e}", exc_info=True)
            # In case of error, return None and do not append to operator_locations to avoid invalid data
            return None


    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables in 2D.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data or on error.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        valid_cables_count = 0
        for c in self.detected_cables:
             if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
                try:
                    # Ensure points are convertible to float
                    start_point_2d = (float(c['start'][0]), float(c['start'][1]))
                    end_point_2d = (float(c['end'][0]), float(c['end'][1]))
                    points_2d.append(start_point_2d)
                    points_2d.append(end_point_2d)
                    valid_cables_count += 1
                except (ValueError, TypeError):
                    logger.warning(f"Skipping cable with invalid start/end coordinates for clustering: {c}")


        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points to form a cluster (excluding noise)
        # and at least 2 points in total to run fit_predict.
        if len(points_2d_np) < max(2, self.corridor_model.min_samples):
            logger.warning(f"Insufficient valid data ({len(points_2d_np)} points from {valid_cables_count} cables) for corridor clustering (requires at least {max(2, self.corridor_model.min_samples)}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            # Optimization: Consider pre-scaling data if coordinates span very different ranges, though unlikely for this data.
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN from {len(points_2d_np)} points.")

            # Return the cluster labels for each point (start and end points interleaved)
            # Mapping labels back to cables (e.g., for per-cable corridor assignment)
            # would require associating the original cable index with the points_2d list creation.
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}", exc_info=True)
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}", exc_info=True)
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        # This function is designed to handle the specific structure of cable dictionaries.
        def serialize_cable_item(cable_dict: Dict[str, Any]) -> Dict[str, Any]:
            serializable_cable = {}
            for k, v in cable_dict.items():
                if k in ['start', 'end'] and isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert point tuples to lists
                elif k == 'length' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float/int
                elif k == 'timestamp' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float
                elif k == 'is_active' and isinstance(v, (bool, np.bool_)):
                     serializable_cable[k] = str(v) # Convert bool/numpy bool to string
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert other numpy arrays to lists
                elif isinstance(v, (int, float, str, type(None))): # Already serializable types
                    serializable_cable[k] = v
                else:
                    # Fallback for potentially unhandled types - convert to string
                    logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' in cable data to string: {v}")
                    serializable_cable[k] = str(v)
            return serializable_cable

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data: Dict[str, Any] = {
            'timestamp': time.time(),
            'detected_cables': [serialize_cable_item(cable) for cable in self.detected_cables], # Use helper for cables
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload).
                # Method: POST
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON).
                # Protocol: UDP
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf).
                # Protocol: AMQP
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis',
                # Secure HTTP endpoint for system health and status (e.g., JSON payload).
                # Method: PUT
                'system_health_status': 'https://defense.system.com/api/v1/status'
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data structure.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.
        # Example:
        # try:
        #     send_to_api(defense_data['integration_endpoints']['cable_detection_api'], defense_data['detected_cables'])
        #     send_to_udp(defense_data['integration_endpoints']['operator_location_protocol'], defense_data['operator_locations'])
        #     publish_message(defense_data['integration_endpoints']['corridor_analysis_service'], defense_data['corridor_clusters'])
        # except Exception as e:
        #     logger.error(f"Failed to send data to defense stack: {e}")
        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should provide mostly serializable data.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj: Any) -> Any:
                if isinstance(obj, dict):
                    # Recursively process dictionary values
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    # Recursively process list elements
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    # Convert tuples to lists (common JSON conversion)
                    return list(obj)
                elif isinstance(obj, np.ndarray):
                    # Convert numpy arrays to lists
                    return obj.tolist()
                elif isinstance(obj, (bool, np.bool_)):
                    # Explicitly convert boolean types (Python bool and numpy bool) to string
                    return str(obj)
                elif isinstance(obj, np.number):
                    # Convert other numpy numbers (int, float) to standard Python types
                    return obj.item() # .item() converts numpy scalar to Python scalar
                # Add other specific type conversions here if needed (e.g., datetime)
                # elif isinstance(obj, datetime.datetime):
                #     return obj.isoformat()
                elif isinstance(obj, (int, float, str, type(None))):
                    # These types are already JSON serializable, return as is
                    return obj
                else:
                    # Fallback for any remaining unhandled types: attempt serialization check
                    # and convert to string if not serializable.
                    try:
                        # Check if it's already serializable by attempting a dump (without logging recursion)
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            # Apply the recursive conversion to the data structure
            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                # Use allow_nan=False for stricter JSON compliance if needed
                json.dump(serializable_data_cleaned, f, indent=2, allow_nan=True)
            logger.info(f"Saved detection data to {filename}")

        except TypeError as e:
            # This catch block should now be less frequent if make_serializable_recursive is robust
            logger.error(f"TypeError during data serialization for saving: {e}", exc_info=True)
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}", exc_info=True)
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}", exc_info=True)


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        # In a real system, initialize hardware only once or manage reconnection logic
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    logger.info(f"Tracing operator for {len(newly_added_cables)} newly added cables...")
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active', default to False
                             estimated_location = self.trace_operator(cable)
                             # In a real system, you might want to handle the case where estimated_location is None
                             # and log/report the failure to trace.
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predicting corridors implicitly
            defense_data_output = self.integrate_with_defense_stack() # Generates data and logs intent
            # Note: Actual sending to defense stack would happen INSIDE integrate_with_defense_stack
            # or be triggered immediately after calling it.

            self.save_data() # Save data to local file for record-keeping/debugging

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        # Example of potentially reading config from a file or env vars
        # detection_range = float(os.getenv('DETECTION_RANGE', 5000.0))
        # min_cable_length = float(os.getenv('MIN_CABLE_LENGTH', 100.0))
        # iterations = int(os.getenv('RUN_ITERATIONS', 5))
        # interval = float(os.getenv('RUN_INTERVAL', 1.0))

        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed due to invalid parameters: {e}", exc_info=True)
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}", exc_info=True)


if __name__ == "__main__":
    # When run as a script, execute main
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 394)

**Reasoning**:
The `IndentationError` persists at the same line number, indicating the indentation issue was not fully resolved in the previous attempt. The error points to an `else` block that is not correctly aligned with its corresponding `if` or `elif`. I will carefully review the indentation in the `save_data` method, particularly within the `make_serializable_recursive` function and around the `else` block indicated in the traceback, and correct it.



In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.

    Attributes:
        detection_range (float): Maximum detection range in meters.
        min_cable_length (float): Minimum cable length to consider for detection.
        detected_cables (List[Dict]): Stores all detected cables over time.
        operator_locations (List[Tuple[float, float]]): Stores estimated operator locations (2D).
        corridor_model (DBSCAN): Clustering model for corridor detection.
        cable_history (List[Dict]): Stores historical cable data for trajectory analysis.
        hardware_initialized (bool): Flag to track hardware initialization status.
        _cable_counter (int): Simple counter for assigning unique IDs to simulated cables.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.

        Raises:
            ValueError: If detection_range or min_cable_length are not positive.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict[str, Any]] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned for density.
        # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        # min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ideally ordered by timestamp.
        self.cable_history: List[Dict[str, Any]] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        This method should contain the actual logic for connecting to and configuring
        the sensor hardware. It's currently a placeholder simulation.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Initialization ---
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Store the connection object if needed for subsequent reads.
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Implement robust error handling and retry mechanisms for connection or configuration failures.
            # --- End Placeholder ---

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}", exc_info=True) # Log exception details
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict[str, Any]]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        This method should contain the actual logic for reading data from the
        initialized sensor hardware. It's currently a placeholder simulation.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or self._hardware_connection is None: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Reading ---
            # 1. Read data from the established hardware connection stream (`self._hardware_connection`).
            #    Example: `raw_sensor_output = self._hardware_connection.read_stream()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols). Parse the raw output.
            #    Example: `parsed_output = parse_sensor_protocol(raw_sensor_output)`
            # 3. Manage buffering of incoming data to ensure complete data packets are processed.
            # 4. Apply sampling rates or data filtering as needed before formatting.
            # 5. Implement robust error handling for data corruption, connection loss during read, or hardware read errors.
            # 6. Convert parsed raw data into the structured format (list of dictionaries)
            #    expected by `process_sensor_data`. Ensure correct data types (float, bool, str) and required keys.
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided unique ID)}
            # --- End Placeholder ---


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT for process_sensor_data
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}", exc_info=True) # Log exception details
            # Consider marking hardware as uninitialized if a critical read error occurs
            # self.hardware_initialized = False
            return []


    def process_sensor_data(self, sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process raw laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List[Dict]: List of dictionaries containing processed cable coordinates and metadata.
                        Output format:
                        {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
                         'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []

        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    missing_keys = [key for key in required_keys if key not in data_point]
                    logger.warning(f"Skipping sensor data point due to missing required keys: {missing_keys}, data: {data_point}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id']) # Ensure cable_id is a string

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue
                except Exception as e:
                     logger.error(f"Unexpected error during type conversion for sensor data point: {e}, data: {data_point}", exc_info=True)
                     continue # Skip this data point on unexpected conversion error


                # Calculate cable length using numpy for efficiency
                # Ensure points are numpy arrays for calculation
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                # Catch any other unexpected errors during processing a single data point
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}", exc_info=True)


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history based on unique ID.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            # Validate presence of cable_id before attempting to add
            cable_id = cable.get('cable_id')
            if cable_id is None:
                 logger.warning(f"Skipping cable detection due to missing 'cable_id': {cable}")
                 continue # Skip this cable if ID is missing

            if cable_id not in detected_cable_ids:
                # Add to detected_cables and history
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.debug(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        # Sorting the entire history repeatedly might become inefficient for very long runs.
        # Consider using a structure that maintains order or sorting a smaller subset if needed.
        if added_count > 0:
             # Ensure 'timestamp' key exists before sorting
             if all('timestamp' in c for c in self.cable_history):
                 self.cable_history.sort(key=lambda x: x['timestamp'])
             else:
                 logger.warning("Skipping history sort: 'timestamp' key missing in some history entries.")

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict[str, Any]) -> Optional[Tuple[float, float]]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently processed cable data
                                 (must include 'start', 'end', 'timestamp').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        # Validate input
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable or 'timestamp' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location_2d: Optional[Tuple[float, float]] = None # Initialize as None

        try:
            curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
            curr_end_2d = np.array(recent_cable['end'][:2], dtype=float) # Ensure float dtype

            # Search for a spatially connected previous cable among recent history
            best_previous_cable = None
            min_distance = float('inf')
            # Tune this threshold based on expected operator movement speed and detection interval
            connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

            # Iterate through history in reverse order of time for recency
            # Limit the search to a reasonable number of recent cables for performance
            # Filter history for valid cables with required keys and timestamp before recent cable
            valid_previous_cables = [
                 c for c in self.cable_history
                 if c.get('timestamp') is not None and c['timestamp'] < recent_cable['timestamp']
                 and 'start' in c and 'end' in c # Ensure start/end are present
            ]
            recent_previous_cables_subset = valid_previous_cables[-50:] # Look at up to the last 50 valid previous cables

            for prev_cable in reversed(recent_previous_cables_subset):
                 prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                 distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                 if distance < connection_threshold: # Found a potential connection
                      if distance < min_distance: # Found a better connection (closer)
                           min_distance = distance
                           best_previous_cable = prev_cable
                      # Optimization: If we find a connection within a very small distance,
                      # assume it's the correct previous segment and stop searching.
                      if distance < 20.0: # Example: within 20m is a strong connection
                           break # Found a very close previous cable, likely the direct predecessor


            if best_previous_cable:
                 # Estimate operator location based on the connected previous cable
                 prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float) # Ensure float dtype
                 time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']

                 if time_diff > 0.1: # Avoid division by zero or very small time differences
                     # Use the direction vector from the end of the previous cable to the start of the current one
                     # as the likely laying direction segment.
                     laying_direction_vector = curr_start_2d - prev_end_2d
                     vector_norm = np.linalg.norm(laying_direction_vector)

                     if vector_norm > 0:
                         estimated_speed = vector_norm / time_diff

                         # Estimate position based on speed and time since the *recent cable* was detected
                         # This assumes the operator continued laying at the estimated speed since the last detection.
                         time_since_recent_detection = time.time() - recent_cable['timestamp'] # Time from cable detection to now
                         extrapolation_distance = estimated_speed * time_since_recent_detection

                         # Limit extrapolation distance to avoid unrealistic jumps
                         max_extrapolation_distance = 500.0 # Example limit, tune based on expected speeds
                         extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)

                         # The operator is estimated to be *behind* the start of the recent cable,
                         # along the *reverse* direction of the laying vector.
                         if vector_norm > 0: # Normalize only if norm is positive
                             normalized_reverse_direction = -laying_direction_vector / vector_norm
                             estimated_operator_pos_2d = curr_start_2d + normalized_reverse_direction * extrapolation_distance
                             operator_location_2d = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats

                             logger.info(f"Estimated operator location based on trajectory with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m (connection distance: {min_distance:.2f}m).")
                         else:
                              logger.warning("Calculated laying direction vector has zero norm. Defaulting to current start point.")
                              operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                     else:
                          logger.warning("Time difference between connected cables is too small (<= 0.1s) or vector norm is zero. Cannot estimate speed/extrapolate. Defaulting to current start point.")
                          operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                else:
                     # Fallback if time_diff is not positive
                     logger.warning("Time difference between connected cables is not positive. Defaulting to current start point.")
                     operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))

            else:
                 # Fallback: If no spatially connected previous cable found, use the start point of the current cable
                 operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Default to current cable start (2D)
                 logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

            # Ensure operator_location_2d is set even in fallbacks
            if operator_location_2d is not None:
                 self.operator_locations.append(operator_location_2d)
                 logger.info(f"Estimated operator location: {operator_location_2d}")
                 return operator_location_2d
            else:
                 # This case should ideally not be reached with the fallbacks, but included for safety.
                 logger.error("Operator location estimation failed unexpectedly.")
                 return None

        except Exception as e:
            logger.error(f"An error occurred during operator trajectory tracing: {e}", exc_info=True)
            # In case of error, return None and do not append to operator_locations to avoid invalid data
            return None


    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables in 2D.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data or on error.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        valid_cables_count = 0
        for c in self.detected_cables:
             if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
                try:
                    # Ensure points are convertible to float
                    start_point_2d = (float(c['start'][0]), float(c['start'][1]))
                    end_point_2d = (float(c['end'][0]), float(c['end'][1]))
                    points_2d.append(start_point_2d)
                    points_2d.append(end_point_2d)
                    valid_cables_count += 1
                except (ValueError, TypeError):
                    logger.warning(f"Skipping cable with invalid start/end coordinates for clustering: {c}")


        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points to form a cluster (excluding noise)
        # and at least 2 points in total to run fit_predict.
        if len(points_2d_np) < max(2, self.corridor_model.min_samples):
            logger.warning(f"Insufficient valid data ({len(points_2d_np)} points from {valid_cables_count} cables) for corridor clustering (requires at least {max(2, self.corridor_model.min_samples)}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            # Optimization: Consider pre-scaling data if coordinates span very different ranges, though unlikely for this data.
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN from {len(points_2d_np)} points.")

            # Return the cluster labels for each point (start and end points interleaved)
            # Mapping labels back to cables (e.g., for per-cable corridor assignment)
            # would require associating the original cable index with the points_2d list creation.
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}", exc_info=True)
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}", exc_info=True)
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        # This function is designed to handle the specific structure of cable dictionaries.
        def serialize_cable_item(cable_dict: Dict[str, Any]) -> Dict[str, Any]:
            serializable_cable = {}
            for k, v in cable_dict.items():
                if k in ['start', 'end'] and isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert point tuples to lists
                elif k == 'length' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float/int
                elif k == 'timestamp' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float
                elif k == 'is_active' and isinstance(v, (bool, np.bool_)):
                     serializable_cable[k] = str(v) # Convert bool/numpy bool to string
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert other numpy arrays to lists
                elif isinstance(v, (int, float, str, type(None))): # Already serializable types
                    serializable_cable[k] = v
                else:
                    # Fallback for potentially unhandled types - convert to string
                    logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' in cable data to string: {v}")
                    serializable_cable[k] = str(v)
            return serializable_cable

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data: Dict[str, Any] = {
            'timestamp': time.time(),
            'detected_cables': [serialize_cable_item(cable) for cable in self.detected_cables], # Use helper for cables
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload).
                # Method: POST
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON).
                # Protocol: UDP
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf).
                # Protocol: AMQP
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis',
                # Secure HTTP endpoint for system health and status (e.g., JSON payload).
                # Method: PUT
                'system_health_status': 'https://defense.system.com/api/v1/status'
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data structure.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.
        # Example:
        # try:
        #     send_to_api(defense_data['integration_endpoints']['cable_detection_api'], defense_data['detected_cables'])
        #     send_to_udp(defense_data['integration_endpoints']['operator_location_protocol'], defense_data['operator_locations'])
        #     publish_message(defense_data['integration_endpoints']['corridor_analysis_service'], defense_data['corridor_clusters'])
        # except Exception as e:
        #     logger.error(f"Failed to send data to defense stack: {e}")
        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should provide mostly serializable data.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj: Any) -> Any:
                if isinstance(obj, dict):
                    # Recursively process dictionary values
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    # Recursively process list elements
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    # Convert tuples to lists (common JSON conversion)
                    return list(obj)
                elif isinstance(obj, np.ndarray):
                    # Convert numpy arrays to lists
                    return obj.tolist()
                elif isinstance(obj, (bool, np.bool_)):
                    # Explicitly convert boolean types (Python bool and numpy bool) to string
                    return str(obj)
                elif isinstance(obj, np.number):
                    # Convert other numpy numbers (int, float) to standard Python types
                    return obj.item() # .item() converts numpy scalar to Python scalar
                # Add other specific type conversions here if needed (e.g., datetime)
                # elif isinstance(obj, datetime.datetime):
                #     return obj.isoformat()
                elif isinstance(obj, (int, float, str, type(None))):
                    # These types are already JSON serializable, return as is
                    return obj
                else:
                    # Fallback for any remaining unhandled types: attempt serialization check
                    # and convert to string if not serializable.
                    try:
                        # Check if it's already serializable by attempting a dump (without logging recursion)
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            # Apply the recursive conversion to the data structure
            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                # Use allow_nan=False for stricter JSON compliance if needed
                json.dump(serializable_data_cleaned, f, indent=2, allow_nan=True)
            logger.info(f"Saved detection data to {filename}")

        except TypeError as e:
            # This catch block should now be less frequent if make_serializable_recursive is robust
            logger.error(f"TypeError during data serialization for saving: {e}", exc_info=True)
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}", exc_info=True)
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}", exc_info=True)


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        # In a real system, initialize hardware only once or manage reconnection logic
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    logger.info(f"Tracing operator for {len(newly_added_cables)} newly added cables...")
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active', default to False
                             estimated_location = self.trace_operator(cable)
                             # In a real system, you might want to handle the case where estimated_location is None
                             # and log/report the failure to trace.
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predicting corridors implicitly
            defense_data_output = self.integrate_with_defense_stack() # Generates data and logs intent
            # Note: Actual sending to defense stack would happen INSIDE integrate_with_defense_stack
            # or be triggered immediately after calling it.

            self.save_data() # Save data to local file for record-keeping/debugging

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        # Example of potentially reading config from a file or env vars
        # detection_range = float(os.getenv('DETECTION_RANGE', 5000.0))
        # min_cable_length = float(os.getenv('MIN_CABLE_LENGTH', 100.0))
        # iterations = int(os.getenv('RUN_ITERATIONS', 5))
        # interval = float(os.getenv('RUN_INTERVAL', 1.0))

        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed due to invalid parameters: {e}", exc_info=True)
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}", exc_info=True)


if __name__ == "__main__":
    # When run as a script, execute main
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 394)

**Reasoning**:
The `IndentationError` at line 394 persists, indicating the issue is still related to the indentation of an `else` block. Given the traceback consistently points to the same line number, it is likely an issue within the `make_serializable_recursive` function inside the `save_data` method. I will regenerate the code, paying extremely careful attention to the indentation within this specific function to ensure all `if`, `elif`, and `else` blocks are correctly aligned.



In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.

    Attributes:
        detection_range (float): Maximum detection range in meters.
        min_cable_length (float): Minimum cable length to consider for detection.
        detected_cables (List[Dict]): Stores all detected cables over time.
        operator_locations (List[Tuple[float, float]]): Stores estimated operator locations (2D).
        corridor_model (DBSCAN): Clustering model for corridor detection.
        cable_history (List[Dict]): Stores historical cable data for trajectory analysis.
        hardware_initialized (bool): Flag to track hardware initialization status.
        _cable_counter (int): Simple counter for assigning unique IDs to simulated cables.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.

        Raises:
            ValueError: If detection_range or min_cable_length are not positive.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict[str, Any]] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned for density.
        # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        # min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ideally ordered by timestamp.
        self.cable_history: List[Dict[str, Any]] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        This method should contain the actual logic for connecting to and configuring
        the sensor hardware. It's currently a placeholder simulation.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Initialization ---
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Store the connection object if needed for subsequent reads.
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Implement robust error handling and retry mechanisms for connection or configuration failures.
            # --- End Placeholder ---

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}", exc_info=True) # Log exception details
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict[str, Any]]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        This method should contain the actual logic for reading data from the
        initialized sensor hardware. It's currently a placeholder simulation.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or self._hardware_connection is None: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Reading ---
            # 1. Read data from the established hardware connection stream (`self._hardware_connection`).
            #    Example: `raw_sensor_output = self._hardware_connection.read_stream()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols). Parse the raw output.
            #    Example: `parsed_output = parse_sensor_protocol(raw_sensor_output)`
            # 3. Manage buffering of incoming data to ensure complete data packets are processed.
            # 4. Apply sampling rates or data filtering as needed before formatting.
            # 5. Implement robust error handling for data corruption, connection loss during read, or hardware read errors.
            # 6. Convert parsed raw data into the structured format (list of dictionaries)
            #    expected by `process_sensor_data`. Ensure correct data types (float, bool, str) and required keys.
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided unique ID)}
            # --- End Placeholder ---


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT for process_sensor_data
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}", exc_info=True) # Log exception details
            # Consider marking hardware as uninitialized if a critical read error occurs
            # self.hardware_initialized = False
            return []


    def process_sensor_data(self, sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process raw laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List[Dict]: List of dictionaries containing processed cable coordinates and metadata.
                        Output format:
                        {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
                         'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []

        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    missing_keys = [key for key in required_keys if key not in data_point]
                    logger.warning(f"Skipping sensor data point due to missing required keys: {missing_keys}, data: {data_point}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id']) # Ensure cable_id is a string

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue
                except Exception as e:
                     logger.error(f"Unexpected error during type conversion for sensor data point: {e}, data: {data_point}", exc_info=True)
                     continue # Skip this data point on unexpected conversion error


                # Calculate cable length using numpy for efficiency
                # Ensure points are numpy arrays for calculation
                cable_length = np.linalg.norm(np.array(end_point) - np.array(start_point))

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                # Catch any other unexpected errors during processing a single data point
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}", exc_info=True)


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history based on unique ID.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            # Validate presence of cable_id before attempting to add
            cable_id = cable.get('cable_id')
            if cable_id is None:
                 logger.warning(f"Skipping cable detection due to missing 'cable_id': {cable}")
                 continue # Skip this cable if ID is missing

            if cable_id not in detected_cable_ids:
                # Add to detected_cables and history
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.debug(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        # Sorting the entire history repeatedly might become inefficient for very long runs.
        # Consider using a structure that maintains order or sorting a smaller subset if needed.
        if added_count > 0:
             # Ensure 'timestamp' key exists before sorting
             if all('timestamp' in c for c in self.cable_history):
                 self.cable_history.sort(key=lambda x: x['timestamp'])
             else:
                 logger.warning("Skipping history sort: 'timestamp' key missing in some history entries.")

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict[str, Any]) -> Optional[Tuple[float, float]]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently processed cable data
                                 (must include 'start', 'end', 'timestamp').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        # Validate input
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable or 'timestamp' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location_2d: Optional[Tuple[float, float]] = None # Initialize as None

        try:
            curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
            curr_end_2d = np.array(recent_cable['end'][:2], dtype=float) # Ensure float dtype

            # Search for a spatially connected previous cable among recent history
            best_previous_cable = None
            min_distance = float('inf')
            # Tune this threshold based on expected operator movement speed and detection interval
            connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

            # Iterate through history in reverse order of time for recency
            # Limit the search to a reasonable number of recent cables for performance
            # Filter history for valid cables with required keys and timestamp before recent cable
            valid_previous_cables = [
                 c for c in self.cable_history
                 if c.get('timestamp') is not None and c['timestamp'] < recent_cable['timestamp']
                 and 'start' in c and 'end' in c # Ensure start/end are present
            ]
            recent_previous_cables_subset = valid_previous_cables[-50:] # Look at up to the last 50 valid previous cables

            for prev_cable in reversed(recent_previous_cables_subset):
                 prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                 distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                 if distance < connection_threshold: # Found a potential connection
                      if distance < min_distance: # Found a better connection (closer)
                           min_distance = distance
                           best_previous_cable = prev_cable
                      # Optimization: If we find a connection within a very small distance,
                      # assume it's the correct previous segment and stop searching.
                      if distance < 20.0: # Example: within 20m is a strong connection
                           break # Found a very close previous cable, likely the direct predecessor


            if best_previous_cable:
                 # Estimate operator location based on the connected previous cable
                 prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float) # Ensure float dtype
                 time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']

                 if time_diff > 0.1: # Avoid division by zero or very small time differences
                     # Use the direction vector from the end of the previous cable to the start of the current one
                     # as the likely laying direction segment.
                     laying_direction_vector = curr_start_2d - prev_end_2d
                     vector_norm = np.linalg.norm(laying_direction_vector)

                     if vector_norm > 0:
                         estimated_speed = vector_norm / time_diff

                         # Estimate position based on speed and time since the *recent cable* was detected
                         # This assumes the operator continued laying at the estimated speed since the last detection.
                         time_since_recent_detection = time.time() - recent_cable['timestamp'] # Time from cable detection to now
                         extrapolation_distance = estimated_speed * time_since_recent_detection

                         # Limit extrapolation distance to avoid unrealistic jumps
                         max_extrapolation_distance = 500.0 # Example limit, tune based on expected speeds
                         extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)

                         # The operator is estimated to be *behind* the start of the recent cable,
                         # along the *reverse* direction of the laying vector.
                         if vector_norm > 0: # Normalize only if norm is positive
                             normalized_reverse_direction = -laying_direction_vector / vector_norm
                             estimated_operator_pos_2d = curr_start_2d + normalized_reverse_direction * extrapolation_distance
                             operator_location_2d = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats

                             logger.info(f"Estimated operator location based on trajectory with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m (connection distance: {min_distance:.2f}m).")
                         else:
                              logger.warning("Calculated laying direction vector has zero norm. Defaulting to current start point.")
                              operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                     else:
                          logger.warning("Time difference between connected cables is too small (<= 0.1s) or vector norm is zero. Cannot estimate speed/extrapolate. Defaulting to current start point.")
                          operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                else:
                     # Fallback if time_diff is not positive
                     logger.warning("Time difference between connected cables is not positive. Defaulting to current start point.")
                     operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))

            else:
                 # Fallback: If no spatially connected previous cable found, use the start point of the current cable
                 operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Default to current cable start (2D)
                 logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

            # Ensure operator_location_2d is set even in fallbacks
            if operator_location_2d is not None:
                 self.operator_locations.append(operator_location_2d)
                 logger.info(f"Estimated operator location: {operator_location_2d}")
                 return operator_location_2d
            else:
                 # This case should ideally not be reached with the fallbacks, but included for safety.
                 logger.error("Operator location estimation failed unexpectedly.")
                 return None

        except Exception as e:
            logger.error(f"An error occurred during operator trajectory tracing: {e}", exc_info=True)
            # In case of error, return None and do not append to operator_locations to avoid invalid data
            return None


    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables in 2D.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data or on error.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        valid_cables_count = 0
        for c in self.detected_cables:
             if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
                try:
                    # Ensure points are convertible to float
                    start_point_2d = (float(c['start'][0]), float(c['start'][1]))
                    end_point_2d = (float(c['end'][0]), float(c['end'][1]))
                    points_2d.append(start_point_2d)
                    points_2d.append(end_point_2d)
                    valid_cables_count += 1
                except (ValueError, TypeError):
                    logger.warning(f"Skipping cable with invalid start/end coordinates for clustering: {c}")


        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points to form a cluster (excluding noise)
        # and at least 2 points in total to run fit_predict.
        if len(points_2d_np) < max(2, self.corridor_model.min_samples):
            logger.warning(f"Insufficient valid data ({len(points_2d_np)} points from {valid_cables_count} cables) for corridor clustering (requires at least {max(2, self.corridor_model.min_samples)}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            # Optimization: Consider pre-scaling data if coordinates span very different ranges, though unlikely for this data.
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN from {len(points_2d_np)} points.")

            # Return the cluster labels for each point (start and end points interleaved)
            # Mapping labels back to cables (e.g., for per-cable corridor assignment)
            # would require associating the original cable index with the points_2d list creation.
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}", exc_info=True)
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}", exc_info=True)
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        # This function is designed to handle the specific structure of cable dictionaries.
        def serialize_cable_item(cable_dict: Dict[str, Any]) -> Dict[str, Any]:
            serializable_cable = {}
            for k, v in cable_dict.items():
                if k in ['start', 'end'] and isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert point tuples to lists
                elif k == 'length' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float/int
                elif k == 'timestamp' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float
                elif k == 'is_active' and isinstance(v, (bool, np.bool_)):
                     serializable_cable[k] = str(v) # Convert bool/numpy bool to string
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert other numpy arrays to lists
                elif isinstance(v, (int, float, str, type(None))): # Already serializable types
                    serializable_cable[k] = v
                else:
                    # Fallback for potentially unhandled types - convert to string
                    logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' in cable data to string: {v}")
                    serializable_cable[k] = str(v)
            return serializable_cable

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data: Dict[str, Any] = {
            'timestamp': time.time(),
            'detected_cables': [serialize_cable_item(cable) for cable in self.detected_cables], # Use helper for cables
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload).
                # Method: POST
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON).
                # Protocol: UDP
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf).
                # Protocol: AMQP
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis',
                # Secure HTTP endpoint for system health and status (e.g., JSON payload).
                # Method: PUT
                'system_health_status': 'https://defense.system.com/api/v1/status'
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data structure.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.
        # Example:
        # try:
        #     send_to_api(defense_data['integration_endpoints']['cable_detection_api'], defense_data['detected_cables'])
        #     send_to_udp(defense_data['integration_endpoints']['operator_location_protocol'], defense_data['operator_locations'])
        #     publish_message(defense_data['integration_endpoints']['corridor_analysis_service'], defense_data['corridor_clusters'])
        # except Exception as e:
        #     logger.error(f"Failed to send data to defense stack: {e}")
        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should provide mostly serializable data.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            def make_serializable_recursive(obj: Any) -> Any:
                if isinstance(obj, dict):
                    # Recursively process dictionary values
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    # Recursively process list elements
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    # Convert tuples to lists (common JSON conversion)
                    return list(obj)
                elif isinstance(obj, np.ndarray):
                    # Convert numpy arrays to lists
                    return obj.tolist()
                elif isinstance(obj, (bool, np.bool_)):
                    # Explicitly convert boolean types (Python bool and numpy bool) to string
                    return str(obj)
                elif isinstance(obj, np.number):
                    # Convert other numpy numbers (int, float) to standard Python types
                    return obj.item() # .item() converts numpy scalar to Python scalar
                # Add other specific type conversions here if needed (e.g., datetime)
                # elif isinstance(obj, datetime.datetime):
                #     return obj.isoformat()
                elif isinstance(obj, (int, float, str, type(None))):
                    # These types are already JSON serializable, return as is
                    return obj
                else:
                    # Fallback for any remaining unhandled types: attempt serialization check
                    # and convert to string if not serializable.
                    try:
                        # Check if it's already serializable by attempting a dump (without logging recursion)
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)

            # Apply the recursive conversion to the data structure
            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                # Use allow_nan=False for stricter JSON compliance if needed
                json.dump(serializable_data_cleaned, f, indent=2, allow_nan=True)
            logger.info(f"Saved detection data to {filename}")

        except TypeError as e:
            # This catch block should now be less frequent if make_serializable_recursive is robust
            logger.error(f"TypeError during data serialization for saving: {e}", exc_info=True)
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}", exc_info=True)
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}", exc_info=True)


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        # In a real system, initialize hardware only once or manage reconnection logic
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    logger.info(f"Tracing operator for {len(newly_added_cables)} newly added cables...")
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active', default to False
                             estimated_location = self.trace_operator(cable)
                             # In a real system, you might want to handle the case where estimated_location is None
                             # and log/report the failure to trace.
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predicting corridors implicitly
            defense_data_output = self.integrate_with_defense_stack() # Generates data and logs intent
            # Note: Actual sending to defense stack would happen INSIDE integrate_with_defense_stack
            # or be triggered immediately after calling it.

            self.save_data() # Save data to local file for record-keeping/debugging

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        # Example of potentially reading config from a file or env vars
        # detection_range = float(os.getenv('DETECTION_RANGE', 5000.0))
        # min_cable_length = float(os.getenv('MIN_CABLE_LENGTH', 100.0))
        # iterations = int(os.getenv('RUN_ITERATIONS', 5))
        # interval = float(os.getenv('RUN_INTERVAL', 1.0))

        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed due to invalid parameters: {e}", exc_info=True)
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}", exc_info=True)


if __name__ == "__main__":
    # When run as a script, execute main
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 394)

**Reasoning**:
The IndentationError persists, consistently pointing to the same line, likely within the `make_serializable_recursive` function in the `save_data` method. Despite previous attempts, the indentation around the `else` block at this specific line is incorrect. I will regenerate the code once more, meticulously checking and correcting the indentation for the `if`, `elif`, and `else` structure within the `make_serializable_recursive` function to resolve this persistent error.



In [7]:
import numpy as np
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.

    Attributes:
        detection_range (float): Maximum detection range in meters.
        min_cable_length (float): Minimum cable length to consider for detection.
        detected_cables (List[Dict]): Stores all detected cables over time.
        operator_locations (List[Tuple[float, float]]): Stores estimated operator locations (2D).
        corridor_model (DBSCAN): Clustering model for corridor detection.
        cable_history (List[Dict]): Stores historical cable data for trajectory analysis.
        hardware_initialized (bool): Flag to track hardware initialization status.
        _cable_counter (int): Simple counter for assigning unique IDs to simulated cables.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.

        Raises:
            ValueError: If detection_range or min_cable_length are not positive.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict[str, Any]] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned for density.
        # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        # min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ideally ordered by timestamp.
        self.cable_history: List[Dict[str, Any]] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        This method should contain the actual logic for connecting to and configuring
        the sensor hardware. It's currently a placeholder simulation.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Initialization ---
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Store the connection object if needed for subsequent reads.
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Implement robust error handling and retry mechanisms for connection or configuration failures.
            # --- End Placeholder ---

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}", exc_info=True) # Log exception details
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict[str, Any]]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        This method should contain the actual logic for reading data from the
        initialized sensor hardware. It's currently a placeholder simulation.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or self._hardware_connection is None: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Reading ---
            # 1. Read data from the established hardware connection stream (`self._hardware_connection`).
            #    Example: `raw_sensor_output = self._hardware_connection.read_stream()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols). Parse the raw output.
            #    Example: `parsed_output = parse_sensor_protocol(raw_sensor_output)`
            # 3. Manage buffering of incoming data to ensure complete data packets are processed.
            # 4. Apply sampling rates or data filtering as needed before formatting.
            # 5. Implement robust error handling for data corruption, connection loss during read, or hardware read errors.
            # 6. Convert parsed raw data into the structured format (list of dictionaries)
            #    expected by `process_sensor_data`. Ensure correct data types (float, bool, str) and required keys.
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided unique ID)}
            # --- End Placeholder ---


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT for process_sensor_data
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}", exc_info=True) # Log exception details
            # Consider marking hardware as uninitialized if a critical read error occurs
            # self.hardware_initialized = False
            return []


    def process_sensor_data(self, sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process raw laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List[Dict]: List of dictionaries containing processed cable coordinates and metadata.
                        Output format:
                        {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
                         'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []

        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    missing_keys = [key for key in required_keys if key not in data_point]
                    logger.warning(f"Skipping sensor data point due to missing required keys: {missing_keys}, data: {data_point}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id']) # Ensure cable_id is a string

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue
                except Exception as e:
                     logger.error(f"Unexpected error during type conversion for sensor data point: {e}, data: {data_point}", exc_info=True)
                     continue # Skip this data point on unexpected conversion error


                # Calculate cable length using numpy for efficiency
                # Ensure points are numpy arrays for calculation
                cable_length = np.linalg.norm(np.array(start_point) - np.array(end_point)) # Corrected order for vector

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                # Catch any other unexpected errors during processing a single data point
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}", exc_info=True)


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history based on unique ID.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            # Validate presence of cable_id before attempting to add
            cable_id = cable.get('cable_id')
            if cable_id is None:
                 logger.warning(f"Skipping cable detection due to missing 'cable_id': {cable}")
                 continue # Skip this cable if ID is missing

            if cable_id not in detected_cable_ids:
                # Add to detected_cables and history
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.debug(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        # Sorting the entire history repeatedly might become inefficient for very long runs.
        # Consider using a structure that maintains order or sorting a smaller subset if needed.
        if added_count > 0:
             # Ensure 'timestamp' key exists before sorting
             if all('timestamp' in c for c in self.cable_history):
                 self.cable_history.sort(key=lambda x: x['timestamp'])
             else:
                 logger.warning("Skipping history sort: 'timestamp' key missing in some history entries.")

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict[str, Any]) -> Optional[Tuple[float, float]]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently processed cable data
                                 (must include 'start', 'end', 'timestamp').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        # Validate input
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable or 'timestamp' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location_2d: Optional[Tuple[float, float]] = None # Initialize as None

        try:
            curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
            # curr_end_2d = np.array(recent_cable['end'][:2], dtype=float) # Ensure float dtype - not used directly in current logic

            # Search for a spatially connected previous cable among recent history
            best_previous_cable = None
            min_distance = float('inf')
            # Tune this threshold based on expected operator movement speed and detection interval
            connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

            # Iterate through history in reverse order of time for recency
            # Limit the search to a reasonable number of recent cables for performance
            # Filter history for valid cables with required keys and timestamp before recent cable
            valid_previous_cables = [
                 c for c in self.cable_history
                 if c.get('timestamp') is not None and c['timestamp'] < recent_cable['timestamp']
                 and 'start' in c and 'end' in c # Ensure start/end are present
            ]
            recent_previous_cables_subset = valid_previous_cables[-50:] # Look at up to the last 50 valid previous cables

            for prev_cable in reversed(recent_previous_cables_subset):
                 prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                 distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                 if distance < connection_threshold: # Found a potential connection
                      if distance < min_distance: # Found a better connection (closer)
                           min_distance = distance
                           best_previous_cable = prev_cable
                      # Optimization: If we find a connection within a very small distance,
                      # assume it's the correct previous segment and stop searching.
                      if distance < 20.0: # Example: within 20m is a strong connection
                           break # Found a very close previous cable, likely the direct predecessor


            if best_previous_cable:
                 # Estimate operator location based on the connected previous cable
                 prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float) # Ensure float dtype
                 time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']

                 if time_diff > 0.1: # Avoid division by zero or very small time differences
                     # Use the direction vector from the end of the previous cable to the start of the current one
                     # as the likely laying direction segment.
                     laying_direction_vector = curr_start_2d - prev_end_2d
                     vector_norm = np.linalg.norm(laying_direction_vector)

                     if vector_norm > 0:
                         estimated_speed = vector_norm / time_diff

                         # Estimate position based on speed and time since the *recent cable* was detected
                         # This assumes the operator continued laying at the estimated speed since the last detection.
                         # It might be more accurate to use time since the *previous* cable was detected,
                         # but using time since the *current* cable was detected assumes the current cable
                         # is the latest information about operator activity.
                         time_since_recent_detection = time.time() - recent_cable['timestamp']
                         # If time_since_recent_detection is negative (clock skew, simulation artifacts),
                         # treat it as 0 or a small positive value.
                         time_since_recent_detection = max(0.0, time_since_recent_detection)


                         extrapolation_distance = estimated_speed * time_since_recent_detection

                         # Limit extrapolation distance to avoid unrealistic jumps
                         max_extrapolation_distance = 500.0 # Example limit, tune based on expected speeds
                         extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)

                         # The operator is estimated to be *behind* the start of the recent cable,
                         # along the *reverse* direction of the laying vector (previous_end to current_start).
                         if vector_norm > 0: # Normalize only if norm is positive
                             normalized_reverse_direction = -laying_direction_vector / vector_norm
                             estimated_operator_pos_2d = curr_start_2d + normalized_reverse_direction * extrapolation_distance
                             operator_location_2d = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats

                             logger.info(f"Estimated operator location based on trajectory with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m (connection distance: {min_distance:.2f}m).")
                         else:
                              logger.warning("Calculated laying direction vector has zero norm. Defaulting to current start point.")
                              operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                     else:
                          logger.warning("Time difference between connected cables is too small (<= 0.1s) or vector norm is zero. Cannot estimate speed/extrapolate. Defaulting to current start point.")
                          operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                else:
                     # Fallback if time_diff is not positive
                     logger.warning("Time difference between connected cables is not positive. Defaulting to current start point.")
                     operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))

            else:
                 # Fallback: If no spatially connected previous cable found, use the start point of the current cable
                 operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Default to current cable start (2D)
                 logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

            # Ensure operator_location_2d is set even in fallbacks
            if operator_location_2d is not None:
                 self.operator_locations.append(operator_location_2d)
                 # logger.debug(f"Appended operator location: {operator_location_2d}")
                 return operator_location_2d
            else:
                 # This case should ideally not be reached with the fallbacks, but included for safety.
                 logger.error("Operator location estimation failed unexpectedly, even with fallbacks.")
                 return None

        except Exception as e:
            logger.error(f"An error occurred during operator trajectory tracing: {e}", exc_info=True)
            # In case of error, return None and do not append to operator_locations to avoid invalid data
            return None


    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables in 2D.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data or on error.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        valid_cables_count = 0
        for c in self.detected_cables:
             if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
                try:
                    # Ensure points are convertible to float
                    start_point_2d = (float(c['start'][0]), float(c['start'][1]))
                    end_point_2d = (float(c['end'][0]), float(c['end'][1]))
                    points_2d.append(start_point_2d)
                    points_2d.append(end_point_2d)
                    valid_cables_count += 1
                except (ValueError, TypeError):
                    logger.warning(f"Skipping cable with invalid start/end coordinates for clustering: {c}")


        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points to form a cluster (excluding noise)
        # and at least 2 points in total to run fit_predict.
        if len(points_2d_np) < max(2, self.corridor_model.min_samples):
            logger.warning(f"Insufficient valid data ({len(points_2d_np)} points from {valid_cables_count} cables) for corridor clustering (requires at least {max(2, self.corridor_model.min_samples)}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            # Optimization: Consider pre-scaling data if coordinates span very different ranges, though unlikely for this data.
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN from {len(points_2d_np)} points.")

            # Return the cluster labels for each point (start and end points interleaved)
            # Mapping labels back to cables (e.g., for per-cable corridor assignment)
            # would require associating the original cable index with the points_2d list creation.
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}", exc_info=True)
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}", exc_info=True)
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        # This function is designed to handle the specific structure of cable dictionaries.
        def serialize_cable_item(cable_dict: Dict[str, Any]) -> Dict[str, Any]:
            serializable_cable = {}
            for k, v in cable_dict.items():
                if k in ['start', 'end'] and isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert point tuples to lists
                elif k == 'length' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float/int
                elif k == 'timestamp' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float
                elif k == 'is_active' and isinstance(v, (bool, np.bool_)):
                     serializable_cable[k] = str(v) # Convert bool/numpy bool to string
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert other numpy arrays to lists
                elif isinstance(v, (int, float, str, type(None))): # Already serializable types
                    serializable_cable[k] = v
                else:
                    # Fallback for potentially unhandled types - convert to string
                    logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' in cable data to string: {v}")
                    serializable_cable[k] = str(v)
            return serializable_cable

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data: Dict[str, Any] = {
            'timestamp': time.time(),
            'detected_cables': [serialize_cable_item(cable) for cable in self.detected_cables], # Use helper for cables
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload).
                # Method: POST
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON).
                # Protocol: UDP
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf).
                # Protocol: AMQP
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis',
                # Secure HTTP endpoint for system health and status (e.g., JSON payload).
                # Method: PUT
                'system_health_status': 'https://defense.system.com/api/v1/status'
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data structure.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.
        # Example:
        # try:
        #     send_to_api(defense_data['integration_endpoints']['cable_detection_api'], defense_data['detected_cables'])
        #     send_to_udp(defense_data['integration_endpoints']['operator_location_protocol'], defense_data['operator_locations'])
        #     publish_message(defense_data['integration_endpoints']['corridor_analysis_service'], defense_data['corridor_clusters'])
        # except Exception as e:
        #     logger.error(f"Failed to send data to defense stack: {e}")
        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should provide mostly serializable data.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            # Ensure consistent indentation for all if/elif/else blocks
            def make_serializable_recursive(obj: Any) -> Any:
                if isinstance(obj, dict):
                    # Recursively process dictionary values
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    # Recursively process list elements
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    # Convert tuples to lists (common JSON conversion)
                    return list(obj)
                elif isinstance(obj, np.ndarray):
                    # Convert numpy arrays to lists
                    return obj.tolist()
                elif isinstance(obj, (bool, np.bool_)):
                    # Explicitly convert boolean types (Python bool and numpy bool) to string
                    return str(obj)
                elif isinstance(obj, np.number):
                    # Convert other numpy numbers (int, float) to standard Python types
                    return obj.item() # .item() converts numpy scalar to Python scalar
                # Add other specific type conversions here if needed (e.g., datetime)
                # elif isinstance(obj, datetime.datetime):
                #     return obj.isoformat()
                elif isinstance(obj, (int, float, str, type(None))):
                    # These types are already JSON serializable, return as is
                    return obj
                else: # Corrected indentation for this else block
                    # Fallback for any remaining unhandled types: attempt serialization check
                    # and convert to string if not serializable.
                    try:
                        # Check if it's already serializable by attempting a dump (without logging recursion)
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)


            # Apply the recursive conversion to the data structure
            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                # Use allow_nan=False for stricter JSON compliance if needed
                json.dump(serializable_data_cleaned, f, indent=2, allow_nan=True)
            logger.info(f"Saved detection data to {filename}")

        except TypeError as e:
            # This catch block should now be less frequent if make_serializable_recursive is robust
            logger.error(f"TypeError during data serialization for saving: {e}", exc_info=True)
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}", exc_info=True)
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}", exc_info=True)


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        # In a real system, initialize hardware only once or manage reconnection logic
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    logger.info(f"Tracing operator for {len(newly_added_cables)} newly added cables...")
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active', default to False
                             estimated_location = self.trace_operator(cable)
                             # In a real system, you might want to handle the case where estimated_location is None
                             # and log/report the failure to trace.
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predicting corridors implicitly
            defense_data_output = self.integrate_with_defense_stack() # Generates data and logs intent
            # Note: Actual sending to defense stack would happen INSIDE integrate_with_defense_stack
            # or be triggered immediately after calling it.

            self.save_data() # Save data to local file for record-keeping/debugging

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

# Modify the main execution block to create a FiberHound instance and call the visualization function
if __name__ == "__main__":
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        # Example of potentially reading config from a file or env vars
        # detection_range = float(os.getenv('DETECTION_RANGE', 5000.0))
        # min_cable_length = float(os.getenv('MIN_CABLE_LENGTH', 100.0))
        # iterations = int(os.getenv('RUN_ITERATIONS', 5))
        # interval = float(os.getenv('RUN_INTERVAL', 1.0))

        # Create an instance of FiberHound
        fiberhound_instance = FiberHound(detection_range=5000.0, min_cable_length=100.0)

        # Run the system simulation
        fiberhound_instance.run(iterations=5, interval=1.0)

        # After the run, visualize the results using the created instance
        # Make sure the visualize_fiberhound_data function is defined in a cell above this one.
        # If the visualization function is in a separate cell, run that cell after this one.
        # visualize_fiberhound_data(fiberhound_instance) # Uncomment this line if visualization function is in the same cell

    except ValueError as e:
        logger.critical(f"System initialization failed due to invalid parameters: {e}", exc_info=True)
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}", exc_info=True)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 402)

**Reasoning**:
The `IndentationError` persists, moving to line 402. This indicates the indentation issue is still within the `make_serializable_recursive` function in `save_data`, specifically around the final `else` block. I will regenerate the code, focusing on the indentation of the final `else` block and its contents within the `make_serializable_recursive` function to definitively resolve this indentation error.



In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
import time
import json
from typing import List, Tuple, Dict, Any, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FiberHound:
    """
    FIBERHOUND system for detecting fibre optic cables, tracing operators,
    predicting drone corridors, and integrating with defense systems.

    Attributes:
        detection_range (float): Maximum detection range in meters.
        min_cable_length (float): Minimum cable length to consider for detection.
        detected_cables (List[Dict]): Stores all detected cables over time.
        operator_locations (List[Tuple[float, float]]): Stores estimated operator locations (2D).
        corridor_model (DBSCAN): Clustering model for corridor detection.
        cable_history (List[Dict]): Stores historical cable data for trajectory analysis.
        hardware_initialized (bool): Flag to track hardware initialization status.
        _cable_counter (int): Simple counter for assigning unique IDs to simulated cables.
    """
    def __init__(self, detection_range: float = 5000.0, min_cable_length: float = 100.0):
        """
        Initialize FIBERHOUND system.

        Args:
            detection_range (float): Maximum detection range in meters.
            min_cable_length (float): Minimum cable length to consider for detection.

        Raises:
            ValueError: If detection_range or min_cable_length are not positive.
        """
        if detection_range <= 0 or min_cable_length <= 0:
            logger.error("Detection range and minimum cable length must be positive.")
            raise ValueError("Detection range and minimum cable length must be positive.")

        self.detection_range = detection_range
        self.min_cable_length = min_cable_length
        self.detected_cables: List[Dict[str, Any]] = [] # Stores all detected cables over time
        self.operator_locations: List[Tuple[float, float]] = []
        # DBSCAN clustering for corridor detection. Parameters can be tuned for density.
        # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        # min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
        self.corridor_model = DBSCAN(eps=50.0, min_samples=3)
        # Store historical cable data for trajectory analysis, ideally ordered by timestamp.
        self.cable_history: List[Dict[str, Any]] = []
        self.hardware_initialized = False # Flag to track hardware initialization status
        # Simple counter for assigning unique IDs to simulated cables
        self._cable_counter = 0
        # Placeholder for actual hardware connection object
        self._hardware_connection = None

    def initialize_hardware(self) -> bool:
        """
        Initialize the real-time laser/acoustic sensor hardware.

        This method should contain the actual logic for connecting to and configuring
        the sensor hardware. It's currently a placeholder simulation.

        Returns:
            bool: True if hardware initialization is successful, False otherwise.
        """
        logger.info("Attempting to initialize sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Initialization ---
            # 1. Identify available hardware interfaces (e.g., serial ports, Ethernet devices).
            #    Example: `hardware_interface = detect_sensor_interface()`
            # 2. Load necessary drivers or libraries for the specific sensor model.
            #    Example: `load_sensor_drivers(hardware_interface)`
            # 3. Establish a connection to the hardware (e.g., open serial port, connect to IP address/port).
            #    Store the connection object if needed for subsequent reads.
            #    Example: `self._hardware_connection = establish_connection(hardware_interface)`
            # 4. Configure hardware settings (e.g., sampling rate, detection parameters, power levels).
            #    Example: `configure_sensor(self._hardware_connection, settings={'rate': 'high'})`
            # 5. Perform self-tests or calibration procedures if available.
            #    Example: `run_calibration(self._hardware_connection)`
            # 6. Implement robust error handling and retry mechanisms for connection or configuration failures.
            # --- End Placeholder ---

            # Simulate initialization success
            time.sleep(0.5) # Simulate time taken for initialization
            self.hardware_initialized = True
            logger.info("Sensor hardware initialization completed (simulated).")
            return True
        except Exception as e:
            logger.error(f"Hardware initialization failed: {e}", exc_info=True) # Log exception details
            self.hardware_initialized = False
            # Optionally, close the connection if it was partially established
            # if self._hardware_connection:
            #     self._hardware_connection.close()
            return False

    def read_from_hardware(self) -> List[Dict[str, Any]]:
        """
        Read raw data streams from the sensor hardware.
        Simulates data in the *expected input format* for `process_sensor_data`.

        This method should contain the actual logic for reading data from the
        initialized sensor hardware. It's currently a placeholder simulation.

        Returns:
            List[Dict]: A list of dictionaries representing raw sensor data points.
                        Returns an empty list if hardware is not initialized or on error.
                        Expected format:
                        {'start_x': float, 'start_y': float, 'start_z': float,
                         'end_x': float, 'end_y': float, 'end_z': float,
                         'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        if not self.hardware_initialized or self._hardware_connection is None: # Check connection object validity
            logger.warning("Hardware not initialized or connection missing. Cannot read data.")
            return []

        logger.info("Reading raw data from sensor hardware...")
        try:
            # --- Placeholder for Real Hardware Reading ---
            # 1. Read data from the established hardware connection stream (`self._hardware_connection`).
            #    Example: `raw_sensor_output = self._hardware_connection.read_stream()`
            # 2. Handle different data formats (e.g., binary, ASCII, specific sensor protocols). Parse the raw output.
            #    Example: `parsed_output = parse_sensor_protocol(raw_sensor_output)`
            # 3. Manage buffering of incoming data to ensure complete data packets are processed.
            # 4. Apply sampling rates or data filtering as needed before formatting.
            # 5. Implement robust error handling for data corruption, connection loss during read, or hardware read errors.
            # 6. Convert parsed raw data into the structured format (list of dictionaries)
            #    expected by `process_sensor_data`. Ensure correct data types (float, bool, str) and required keys.
            #    {'start_x': float, 'start_y': float, 'start_z': float,
            #     'end_x': float, 'end_y': float, 'end_z': float,
            #     'timestamp': float, 'is_active': bool, 'cable_id': str (or sensor-provided unique ID)}
            # --- End Placeholder ---


            # Simulate reading dummy sensor data in the CORRECT INPUT FORMAT for process_sensor_data
            dummy_sensor_data = []
            num_simulated_detections = np.random.randint(1, 10) # Simulate variable sensor input
            current_time = time.time() # Use a consistent timestamp base for one read cycle
            for i in range(num_simulated_detections):
                 start_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), 0)
                 end_point = (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000), np.random.uniform(10, 100))
                 self._cable_counter += 1 # Assign a simple unique ID
                 # Simulate slightly different timestamps within the same read for realism
                 event_timestamp = current_time - (num_simulated_detections - 1 - i) * 0.01 # Simulate events occurring close in time

                 dummy_sensor_data.append({
                     'start_x': float(start_point[0]), 'start_y': float(start_point[1]), 'start_z': float(start_point[2]),
                     'end_x': float(end_point[0]), 'end_y': float(end_point[1]), 'end_z': float(end_point[2]),
                     'timestamp': event_timestamp,
                     'is_active': bool(np.random.choice([True, False], p=[0.8, 0.2])),
                     'cable_id': f'sim-cable-{self._cable_counter}' # Simulate a unique ID
                 })
            logger.info(f"Read {len(dummy_sensor_data)} raw data points from hardware (simulated, correct format).")
            return dummy_sensor_data
        except Exception as e:
            logger.error(f"Error reading from hardware: {e}", exc_info=True) # Log exception details
            # Consider marking hardware as uninitialized if a critical read error occurs
            # self.hardware_initialized = False
            return []


    def process_sensor_data(self, sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process raw laser/acoustic sensor data for fibre optic cables.
        Validates format, calculates length, and filters by min_cable_length.

        Args:
            sensor_data (List[Dict]): A list of dictionaries, representing raw sensor data points.
                                      Expected format for each dictionary:
                                      {'start_x': float, 'start_y': float, 'start_z': float,
                                       'end_x': float, 'end_y': float, 'end_z': float,
                                       'timestamp': float, 'is_active': bool, 'cable_id': str}

        Returns:
            List[Dict]: List of dictionaries containing processed cable coordinates and metadata.
                        Output format:
                        {'start': Tuple[float, float, float], 'end': Tuple[float, float, float],
                         'length': float, 'timestamp': float, 'is_active': bool, 'cable_id': str}
        """
        processed_cables = []

        for data_point in sensor_data:
            try:
                # Validate required keys are present
                required_keys = ['start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z', 'timestamp', 'is_active', 'cable_id']
                if not all(key in data_point for key in required_keys):
                    missing_keys = [key for key in required_keys if key not in data_point]
                    logger.warning(f"Skipping sensor data point due to missing required keys: {missing_keys}, data: {data_point}")
                    continue

                # Basic type validation and conversion
                try:
                    start_point = (float(data_point['start_x']), float(data_point['start_y']), float(data_point['start_z']))
                    end_point = (float(data_point['end_x']), float(data_point['end_y']), float(data_point['end_z']))
                    timestamp = float(data_point['timestamp'])
                    is_active = bool(data_point['is_active'])
                    cable_id = str(data_point['cable_id']) # Ensure cable_id is a string

                except (ValueError, TypeError) as e:
                    logger.warning(f"Skipping sensor data point due to invalid type conversion: {e}, data: {data_point}")
                    continue
                except Exception as e:
                     logger.error(f"Unexpected error during type conversion for sensor data point: {e}, data: {data_point}", exc_info=True)
                     continue # Skip this data point on unexpected conversion error


                # Calculate cable length using numpy for efficiency
                # Ensure points are numpy arrays for calculation
                cable_length = np.linalg.norm(np.array(start_point) - np.array(end_point)) # Corrected order for vector

                if cable_length >= self.min_cable_length:
                    processed_cables.append({
                        'start': start_point,
                        'end': end_point,
                        'length': float(cable_length), # Ensure float
                        'timestamp': timestamp,
                        'is_active': is_active,
                        'cable_id': cable_id
                    })
            except Exception as e:
                # Catch any other unexpected errors during processing a single data point
                logger.error(f"Unexpected error processing sensor data point: {e}, data: {data_point}", exc_info=True)


        logger.info(f"Processed {len(processed_cables)} valid cable detections from sensor data")
        return processed_cables

    def detect_cables(self, processed_sensor_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Detect fibre optic cables from processed sensor data.
        Adds *newly detected* cables to the internal list and history based on unique ID.

        Args:
            processed_sensor_data (List[Dict]): List of processed cable data dictionaries.

        Returns:
             List[Dict]: A list of the cables that were newly added in this detection cycle.
        """
        if not processed_sensor_data:
            logger.info("No processed data to detect cables from.")
            return []

        newly_added_cables = []
        # Use a set for efficient checking of already detected cables based on cable_id
        # Assuming cable_id is a unique identifier provided by the sensor or processing.
        detected_cable_ids = {c.get('cable_id') for c in self.detected_cables if 'cable_id' in c}

        added_count = 0
        for cable in processed_sensor_data:
            # Validate presence of cable_id before attempting to add
            cable_id = cable.get('cable_id')
            if cable_id is None:
                 logger.warning(f"Skipping cable detection due to missing 'cable_id': {cable}")
                 continue # Skip this cable if ID is missing

            if cable_id not in detected_cable_ids:
                # Add to detected_cables and history
                self.detected_cables.append(cable)
                self.cable_history.append(cable)
                newly_added_cables.append(cable)
                detected_cable_ids.add(cable_id) # Add to set for quick lookup
                added_count += 1
                # logger.debug(f"Detected new cable: ID={cable_id}, start={cable['start']}, end={cable['end']}, active={cable['is_active']}")
            # else:
                # logger.debug(f"Cable with ID {cable_id} already detected.")

        logger.info(f"Added {added_count} new unique cables to detected list and history.")

        # Keep history sorted by timestamp for trajectory analysis (Optimization: only sort if items were added)
        # Sorting the entire history repeatedly might become inefficient for very long runs.
        # Consider using a structure that maintains order or sorting a smaller subset if needed.
        if added_count > 0:
             # Ensure 'timestamp' key exists before sorting
             if all('timestamp' in c for c in self.cable_history):
                 self.cable_history.sort(key=lambda x: x['timestamp'])
             else:
                 logger.warning("Skipping history sort: 'timestamp' key missing in some history entries.")

        return newly_added_cables


    def trace_operator(self, recent_cable: Dict[str, Any]) -> Optional[Tuple[float, float]]:
        """
        Trace recent cable detection back to estimated operator location using trajectory analysis.
        This is a placeholder for more sophisticated analysis.

        Args:
            recent_cable (Dict): The most recently processed cable data
                                 (must include 'start', 'end', 'timestamp').

        Returns:
            Tuple[float, float]: Estimated (x, y) coordinates of operator.
                                 Returns None if tracing is not possible or fails.
        """
        # Validate input
        if not recent_cable or 'start' not in recent_cable or 'end' not in recent_cable or 'timestamp' not in recent_cable:
            logger.warning("Cannot trace operator: invalid or incomplete recent cable data provided.")
            return None

        logger.info(f"Tracing operator for recent cable: ID={recent_cable.get('cable_id', 'N/A')}")

        # --- Sophisticated Trajectory Analysis Placeholder ---
        # This implementation refines the previous heuristic approach.
        # A real implementation would involve:
        # 1. Analyzing the temporal and spatial relationship of recent cables in self.cable_history.
        # 2. Identifying sequences of cables likely laid by the same operator (e.g., based on proximity, time gaps).
        # 3. Estimating laying direction, speed, and potential pauses or changes in direction.
        # 4. Using filtering techniques (e.g., Kalman filters) to smooth location estimates over time.
        # 5. Incorporating external data (e.g., GPS data if available, terrain information).

        # Current Placeholder Logic Refinement:
        # - Find the cable in history that is spatially closest at its end to the start of the recent cable,
        #   and is temporally before the recent cable.
        # - If a connected previous cable is found, estimate the operator location by extrapolating
        #   backward from the current cable's start point along the direction vector from the previous cable's end
        #   to the current cable's start.
        # - The extrapolation distance is a heuristic based on the estimated laying speed.
        # - If no connected previous cable is found, default to the start of the current cable (as a fallback).

        operator_location_2d: Optional[Tuple[float, float]] = None # Initialize as None

        try:
            curr_start_2d = np.array(recent_cable['start'][:2], dtype=float) # Ensure float dtype
            # curr_end_2d = np.array(recent_cable['end'][:2], dtype=float) # Ensure float dtype - not used directly in current logic

            # Search for a spatially connected previous cable among recent history
            best_previous_cable = None
            min_distance = float('inf')
            # Tune this threshold based on expected operator movement speed and detection interval
            connection_threshold = 200.0 # Max distance between end of previous and start of current to consider connected

            # Iterate through history in reverse order of time for recency
            # Limit the search to a reasonable number of recent cables for performance
            # Filter history for valid cables with required keys and timestamp before recent cable
            valid_previous_cables = [
                 c for c in self.cable_history
                 if c.get('timestamp') is not None and c['timestamp'] < recent_cable['timestamp']
                 and 'start' in c and 'end' in c # Ensure start/end are present
            ]
            recent_previous_cables_subset = valid_previous_cables[-50:] # Look at up to the last 50 valid previous cables

            for prev_cable in reversed(recent_previous_cables_subset):
                 prev_end_2d = np.array(prev_cable['end'][:2], dtype=float) # Ensure float dtype
                 distance = np.linalg.norm(curr_start_2d - prev_end_2d)

                 if distance < connection_threshold: # Found a potential connection
                      if distance < min_distance: # Found a better connection (closer)
                           min_distance = distance
                           best_previous_cable = prev_cable
                      # Optimization: If we find a connection within a very small distance,
                      # assume it's the correct previous segment and stop searching.
                      if distance < 20.0: # Example: within 20m is a strong connection
                           break # Found a very close previous cable, likely the direct predecessor


            if best_previous_cable:
                 # Estimate operator location based on the connected previous cable
                 prev_end_2d = np.array(best_previous_cable['end'][:2], dtype=float) # Ensure float dtype
                 time_diff = recent_cable['timestamp'] - best_previous_cable['timestamp']

                 if time_diff > 0.1: # Avoid division by zero or very small time differences
                     # Use the direction vector from the end of the previous cable to the start of the current one
                     # as the likely laying direction segment.
                     laying_direction_vector = curr_start_2d - prev_end_2d
                     vector_norm = np.linalg.norm(laying_direction_vector)

                     if vector_norm > 0:
                         estimated_speed = vector_norm / time_diff

                         # Estimate position based on speed and time since the *recent cable* was detected
                         # This assumes the operator continued laying at the estimated speed since the last detection.
                         # It might be more accurate to use time since the *previous* cable was detected,
                         # but using time since the *current* cable was detected assumes the current cable
                         # is the latest information about operator activity.
                         time_since_recent_detection = time.time() - recent_cable['timestamp']
                         # If time_since_recent_detection is negative (clock skew, simulation artifacts),
                         # treat it as 0 or a small positive value.
                         time_since_recent_detection = max(0.0, time_since_recent_detection)


                         extrapolation_distance = estimated_speed * time_since_recent_detection

                         # Limit extrapolation distance to avoid unrealistic jumps
                         max_extrapolation_distance = 500.0 # Example limit, tune based on expected speeds
                         extrapolation_distance = min(extrapolation_distance, max_extrapolation_distance)

                         # The operator is estimated to be *behind* the start of the recent cable,
                         # along the *reverse* direction of the laying vector (previous_end to current_start).
                         if vector_norm > 0: # Normalize only if norm is positive
                             normalized_reverse_direction = -laying_direction_vector / vector_norm
                             estimated_operator_pos_2d = curr_start_2d + normalized_reverse_direction * extrapolation_distance
                             operator_location_2d = (float(estimated_operator_pos_2d[0]), float(estimated_operator_pos_2d[1])) # Ensure floats

                             logger.info(f"Estimated operator location based on trajectory with previous cable ID {best_previous_cable.get('cable_id', 'N/A')}. Estimated speed: {estimated_speed:.2f} m/s. Extrapolated distance: {extrapolation_distance:.2f} m (connection distance: {min_distance:.2f}m).")
                         else:
                              logger.warning("Calculated laying direction vector has zero norm. Defaulting to current start point.")
                              operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                     else:
                          logger.warning("Time difference between connected cables is too small (<= 0.1s) or vector norm is zero. Cannot estimate speed/extrapolate. Defaulting to current start point.")
                          operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))
                else:
                     # Fallback if time_diff is not positive
                     logger.warning("Time difference between connected cables is not positive. Defaulting to current start point.")
                     operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1]))

            else:
                 # Fallback: If no spatially connected previous cable found, use the start point of the current cable
                 operator_location_2d = (float(curr_start_2d[0]), float(curr_start_2d[1])) # Default to current cable start (2D)
                 logger.info(f"No spatially connected previous cable found within {connection_threshold}m among recent history, defaulting to current start point.")

            # Ensure operator_location_2d is set even in fallbacks
            if operator_location_2d is not None:
                 self.operator_locations.append(operator_location_2d)
                 # logger.debug(f"Appended operator location: {operator_location_2d}")
                 return operator_location_2d
            else:
                 # This case should ideally not be reached with the fallbacks, but included for safety.
                 logger.error("Operator location estimation failed unexpectedly, even with fallbacks.")
                 return None

        except Exception as e:
            logger.error(f"An error occurred during operator trajectory tracing: {e}", exc_info=True)
            # In case of error, return None and do not append to operator_locations to avoid invalid data
            return None


    def predict_corridors(self) -> np.ndarray:
        """
        Use clustering (DBSCAN) to identify preferred drone corridors based on cable trajectories.
        Clusters points representing the start and end of detected cables in 2D.

        Returns:
            np.ndarray: Cluster labels for each point (start/end). Returns empty array if insufficient data or on error.
        """
        if not self.detected_cables:
            logger.warning("No cables detected for corridor prediction")
            return np.array([])

        # Extract 2D start and end points for clustering
        # Each cable contributes two points (start_x, start_y) and (end_x, end_y)
        points_2d = []
        valid_cables_count = 0
        for c in self.detected_cables:
             if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
                try:
                    # Ensure points are convertible to float
                    start_point_2d = (float(c['start'][0]), float(c['start'][1]))
                    end_point_2d = (float(c['end'][0]), float(c['end'][1]))
                    points_2d.append(start_point_2d)
                    points_2d.append(end_point_2d)
                    valid_cables_count += 1
                except (ValueError, TypeError):
                    logger.warning(f"Skipping cable with invalid start/end coordinates for clustering: {c}")


        points_2d_np = np.array(points_2d)

        # DBSCAN requires at least min_samples points to form a cluster (excluding noise)
        # and at least 2 points in total to run fit_predict.
        if len(points_2d_np) < max(2, self.corridor_model.min_samples):
            logger.warning(f"Insufficient valid data ({len(points_2d_np)} points from {valid_cables_count} cables) for corridor clustering (requires at least {max(2, self.corridor_model.min_samples)}).")
            # Return an array of -1s indicating all points are noise due to insufficient data
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])

        try:
            # Perform DBSCAN clustering
            # Optimization: Consider pre-scaling data if coordinates span very different ranges, though unlikely for this data.
            clusters = self.corridor_model.fit_predict(points_2d_np)

            # Note: DBSCAN assigns -1 to noise points.
            num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
            logger.info(f"Identified {num_clusters} corridors using DBSCAN from {len(points_2d_np)} points.")

            # Return the cluster labels for each point (start and end points interleaved)
            # Mapping labels back to cables (e.g., for per-cable corridor assignment)
            # would require associating the original cable index with the points_2d list creation.
            return clusters
        except Exception as e:
            logger.error(f"Error during corridor clustering: {e}", exc_info=True)
            # Return an array of -1s on error, matching the number of points
            return np.full(len(points_2d_np), -1) if len(points_2d_np) > 0 else np.array([])


    def integrate_with_defense_stack(self) -> Dict[str, Any]:
        """
        Prepare and structure detection data for integration with a layered defense stack.
        Includes placeholder API endpoints or communication protocols.
        Ensures data is JSON serializable.

        Returns:
            Dict: Structured data for defense stack integration.
        """
        try:
            # Get corridor clusters and ensure they are lists of serializable types
            corridor_clusters_raw = self.predict_corridors()
            corridor_clusters = corridor_clusters_raw.tolist() if isinstance(corridor_clusters_raw, np.ndarray) else []
        except Exception as e:
            logger.error(f"Error getting corridor clusters for integration: {e}", exc_info=True)
            corridor_clusters = [] # Provide empty list on error

        # Helper function to make individual cable/location dictionaries serializable
        # This function is designed to handle the specific structure of cable dictionaries.
        def serialize_cable_item(cable_dict: Dict[str, Any]) -> Dict[str, Any]:
            serializable_cable = {}
            for k, v in cable_dict.items():
                if k in ['start', 'end'] and isinstance(v, tuple):
                    serializable_cable[k] = list(v) # Convert point tuples to lists
                elif k == 'length' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float/int
                elif k == 'timestamp' and isinstance(v, np.number):
                     serializable_cable[k] = float(v) # Convert numpy float/int to standard float
                elif k == 'is_active' and isinstance(v, (bool, np.bool_)):
                     serializable_cable[k] = str(v) # Convert bool/numpy bool to string
                elif isinstance(v, np.ndarray):
                     serializable_cable[k] = v.tolist() # Convert other numpy arrays to lists
                elif isinstance(v, (int, float, str, type(None))): # Already serializable types
                    serializable_cable[k] = v
                else:
                    # Fallback for potentially unhandled types - convert to string
                    logger.warning(f"Converting potentially non-serializable value of type {type(v)} for key '{k}' in cable data to string: {v}")
                    serializable_cable[k] = str(v)
            return serializable_cable

        # Convert operator_locations tuples to lists for JSON serialization
        operator_locations_serializable = [list(loc) if isinstance(loc, tuple) else loc for loc in self.operator_locations] # Ensure locations are lists


        defense_data: Dict[str, Any] = {
            'timestamp': time.time(),
            'detected_cables': [serialize_cable_item(cable) for cable in self.detected_cables], # Use helper for cables
            'operator_locations': operator_locations_serializable,
            'corridor_clusters': corridor_clusters,
            'integration_endpoints': {
                # REST API endpoint for sending new cable detections (e.g., JSON payload).
                # Method: POST
                'cable_detection_api': 'https://defense.system.com/api/v1/cable_detections',
                # UDP endpoint for sending real-time operator location updates (e.g., binary or JSON).
                # Protocol: UDP
                'operator_location_protocol': 'UDP://192.168.1.100:5005',
                # Message Queue topic for publishing corridor analysis results (e.g., JSON or Protobuf).
                # Protocol: AMQP
                'corridor_analysis_service': 'AMQP://message.queue.com/topic/corridor_analysis',
                # Secure HTTP endpoint for system health and status (e.g., JSON payload).
                # Method: PUT
                'system_health_status': 'https://defense.system.com/api/v1/status'
            },
            # Ensure system_status is a string for JSON serialization
            'system_status': 'Operational' if self.hardware_initialized else 'Hardware Error'
        }
        logger.info("Generated defense stack integration data structure.")

        # --- Placeholder for Real Defense Stack Communication ---
        # In a real system, you would add code here to SEND this data
        # to the actual endpoints using appropriate libraries (e.g., requests, socket, pika).
        # This would likely involve asynchronous calls or a dedicated sender thread/process.
        # Example:
        # try:
        #     send_to_api(defense_data['integration_endpoints']['cable_detection_api'], defense_data['detected_cables'])
        #     send_to_udp(defense_data['integration_endpoints']['operator_location_protocol'], defense_data['operator_locations'])
        #     publish_message(defense_data['integration_endpoints']['corridor_analysis_service'], defense_data['corridor_clusters'])
        # except Exception as e:
        #     logger.error(f"Failed to send data to defense stack: {e}")
        # --- End Placeholder ---

        return defense_data

    def save_data(self, filename: str = "fiberhound_data.json") -> None:
        """
        Save detection data to a local file for offline analysis.
        Ensures data is JSON serializable by converting non-serializable types.

        Args:
            filename (str): Output file name.
        """
        try:
            # Get the data structure. integrate_with_defense_stack should provide mostly serializable data.
            data_to_save = self.integrate_with_defense_stack()

            # Final robust recursive function to catch any remaining non-serializable types
            # Ensure consistent indentation for all if/elif/else blocks
            def make_serializable_recursive(obj: Any) -> Any:
                if isinstance(obj, dict):
                    # Recursively process dictionary values
                    return {k: make_serializable_recursive(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    # Recursively process list elements
                    return [make_serializable_recursive(elem) for elem in obj]
                elif isinstance(obj, tuple):
                    # Convert tuples to lists (common JSON conversion)
                    return list(obj)
                elif isinstance(obj, np.ndarray):
                    # Convert numpy arrays to lists
                    return obj.tolist()
                elif isinstance(obj, (bool, np.bool_)):
                    # Explicitly convert boolean types (Python bool and numpy bool) to string
                    return str(obj)
                elif isinstance(obj, np.number):
                    # Convert other numpy numbers (int, float) to standard Python types
                    return obj.item() # .item() converts numpy scalar to Python scalar
                # Add other specific type conversions here if needed (e.g., datetime)
                # elif isinstance(obj, datetime.datetime):
                #     return obj.isoformat()
                elif isinstance(obj, (int, float, str, type(None))):
                    # These types are already JSON serializable, return as is
                    return obj
                # --- Start of section around line 402 ---
                else:
                    # Fallback for any remaining unhandled types: attempt serialization check
                    # and convert to string if not serializable.
                    try:
                        # Check if it's already serializable by attempting a dump (without logging recursion)
                        json.dumps(obj)
                        return obj
                    except TypeError:
                        # If not serializable, log a warning and convert to string
                        logger.warning(f"Converting potentially non-serializable type {type(obj)} to string for saving: {obj}")
                        return str(obj)
                # --- End of section around line 402 ---

            # Apply the recursive conversion to the data structure
            serializable_data_cleaned = make_serializable_recursive(data_to_save)

            with open(filename, 'w') as f:
                # Use allow_nan=False for stricter JSON compliance if needed
                json.dump(serializable_data_cleaned, f, indent=2, allow_nan=True)
            logger.info(f"Saved detection data to {filename}")

        except TypeError as e:
            # This catch block should now be less frequent if make_serializable_recursive is robust
            logger.error(f"TypeError during data serialization for saving: {e}", exc_info=True)
        except IOError as e:
            logger.error(f"IOError during data saving to {filename}: {e}", exc_info=True)
        except Exception as e:
            logger.error(f"An unexpected error occurred during data saving: {e}", exc_info=True)


    def run(self, iterations: int = 5, interval: float = 1.0) -> None:
        """
        Run FIBERHOUND system simulation, integrating with simulated hardware reads.

        Args:
            iterations (int): Number of detection cycles.
            interval (float): Time interval between detections in seconds.
        """
        logger.info("Starting FIBERHOUND system run.")

        # Step 1: Initialize hardware at the beginning
        # In a real system, initialize hardware only once or manage reconnection logic
        if not self.initialize_hardware():
            logger.error("Hardware initialization failed. Aborting run.")
            return

        for i in range(iterations):
            logger.info(f"--- Processing iteration {i+1}/{iterations} ---")

            # Step 2: Read data from hardware (simulated for now, in correct input format)
            raw_sensor_data = self.read_from_hardware()

            if raw_sensor_data:
                # Step 3: Process raw sensor data into structured cable data
                processed_cables_this_cycle = self.process_sensor_data(raw_sensor_data)

                # Step 4: Detect unique cables from the processed data and add to history
                newly_added_cables = self.detect_cables(processed_cables_this_cycle)

                # Step 5: Trace operator for newly added active cables
                if newly_added_cables:
                    logger.info(f"Tracing operator for {len(newly_added_cables)} newly added cables...")
                    for cable in newly_added_cables:
                         if cable.get('is_active', False): # Safely check 'is_active', default to False
                             estimated_location = self.trace_operator(cable)
                             # In a real system, you might want to handle the case where estimated_location is None
                             # and log/report the failure to trace.
                else:
                    logger.info("No new cables added this iteration, skipping operator tracing.")


            else:
                logger.warning(f"No valid data processed from hardware read in iteration {i+1}")

            # Step 6: Integrate with defense stack and save data
            # integrate_with_defense_stack includes predicting corridors implicitly
            defense_data_output = self.integrate_with_defense_stack() # Generates data and logs intent
            # Note: Actual sending to defense stack would happen INSIDE integrate_with_defense_stack
            # or be triggered immediately after calling it.

            self.save_data() # Save data to local file for record-keeping/debugging

            logger.info(f"--- Iteration {i+1}/{iterations} completed ---")

            if i < iterations - 1: # Don't sleep after the last iteration
                time.sleep(interval)

        # --- Placeholder for Hardware Deinitialization ---
        # In a real system, you might want to deinitialize hardware when the run stops.
        # try:
        #     self._hardware_connection.close() # Example close
        #     logger.info("Hardware connection closed.")
        # except Exception as e:
        #     logger.error(f"Error closing hardware connection: {e}")
        # --- End Placeholder ---

        logger.info("FIBERHOUND system run completed.")

def main():
    # Initialize and run FIBERHOUND system
    # Consider using environment variables or config files for parameters in production
    try:
        # Example of potentially reading config from a file or env vars
        # detection_range = float(os.getenv('DETECTION_RANGE', 5000.0))
        # min_cable_length = float(os.getenv('MIN_CABLE_LENGTH', 100.0))
        # iterations = int(os.getenv('RUN_ITERATIONS', 5))
        # interval = float(os.getenv('RUN_INTERVAL', 1.0))

        fiberhound = FiberHound(detection_range=5000.0, min_cable_length=100.0)
        fiberhound.run(iterations=5, interval=1.0)
    except ValueError as e:
        logger.critical(f"System initialization failed due to invalid parameters: {e}", exc_info=True)
    except Exception as e:
        logger.critical(f"An unhandled error occurred during system execution: {e}", exc_info=True)


if __name__ == "__main__":
    # When run as a script, execute main
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 402)

## Summary:

### Data Analysis Key Findings

*   The `process_sensor_data` method was refactored to accept real sensor data (simulated), including validation for required keys and basic type conversion.
*   The `trace_operator` method was enhanced to use a more sophisticated heuristic for estimating operator location by analyzing the spatial and temporal relationships of connected cable segments in the history.
*   Placeholder methods (`initialize_hardware`, `read_from_hardware`) were added with detailed comments outlining the steps for real hardware integration.
*   The `integrate_with_defense_stack` method was updated to include placeholder API endpoints and communication protocols (HTTP, UDP, AMQP) for different data types.
*   Robust JSON serialization logic was implemented, particularly in the `save_data` method, using a recursive helper function to handle tuples, NumPy arrays, and boolean types.

### Insights or Next Steps

*   The persistent `IndentationError` indicates a syntax issue that needs to be resolved to make the code executable and verify all implemented improvements.
*   Future steps should focus on resolving the indentation error and then implementing the actual communication logic within `integrate_with_defense_stack` using appropriate libraries for each protocol.


### Visualize Results

Let's visualize the detected cables, estimated operator locations, and predicted corridors using Matplotlib.

In [3]:
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch # For plotting cables as arrows

def visualize_fiberhound_data(fiberhound_instance: FiberHound):
    """
    Visualizes the detected cables, operator locations, and predicted corridors.

    Args:
        fiberhound_instance (FiberHound): An instance of the FiberHound class
                                          containing detected data.
    """
    plt.figure(figsize=(12, 8))
    ax = plt.gca() # Get current axes

    # Visualize Detected Cables
    if fiberhound_instance.detected_cables:
        logger.info("Visualizing detected cables...")
        for cable in fiberhound_instance.detected_cables:
            start = cable['start']
            end = cable['end']
            is_active = cable.get('is_active', False)
            cable_id = cable.get('cable_id', 'N/A')

            # Use FancyArrowPatch to show direction from start to end
            arrow = FancyArrowPatch(
                (start[0], start[1]), (end[0], end[1]),
                mutation_scale=10, # Adjust arrow size
                lw=1.5, # Line width
                arrowstyle="-|>", # Arrow style
                color='blue' if is_active else 'gray', # Blue for active, gray for inactive
                linestyle='-', # Solid line
                alpha=0.7, # Semi-transparent
                label='Active Cable' if is_active and not ax.yaxis.get_label() else 'Inactive Cable' if not is_active and not ax.xaxis.get_label() else "" # Add label once
            )
            ax.add_patch(arrow)

    # Visualize Operator Locations
    if fiberhound_instance.operator_locations:
        logger.info("Visualizing operator locations...")
        # Plot operator locations as red circles
        op_x = [loc[0] for loc in fiberhound_instance.operator_locations]
        op_y = [loc[1] for loc in fiberhound_instance.operator_locations]
        ax.scatter(op_x, op_y, color='red', marker='o', s=50, label='Operator Location', zorder=5) # zorder to ensure visibility


    # Visualize Corridors (DBSCAN clusters)
    clusters = fiberhound_instance.predict_corridors()
    points_for_clustering = []
    for c in fiberhound_instance.detected_cables:
        if 'start' in c and 'end' in c and len(c['start']) >= 2 and len(c['end']) >= 2:
             try:
                 points_for_clustering.append(c['start'][:2])
                 points_for_clustering.append(c['end'][:2])
             except (ValueError, TypeError):
                 pass # Skip invalid points, already logged in predict_corridors

    if points_for_clustering and clusters is not None and len(clusters) == len(points_for_clustering):
         logger.info("Visualizing corridors...")
         points_2d_np = np.array(points_for_clustering)

         # Get unique labels, excluding noise (-1)
         unique_labels = set(clusters)
         colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

         for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]
                marker = 'x' # Use 'x' for noise points
                label = 'Noise'
            else:
                marker = '.' # Use '.' for clustered points
                label = f'Corridor {k}'

            # Create a boolean mask for points belonging to the current cluster
            class_member_mask = (clusters == k)

            # Plot the core points in this cluster (DBSCAN doesn't explicitly define core points in labels,
            # but we plot all points belonging to the cluster, noise points are handled separately).
            xy = points_2d_np[class_member_mask]
            ax.plot(xy[:, 0], xy[:, 1], marker, markerfacecolor=tuple(col),
                    markeredgecolor=tuple(col), markersize=6, linestyle='None', label=label if k != -1 else None) # Add label once for clustered points, None for noise


    # Set plot title and labels
    ax.set_title('FIBERHOUND Detection Visualization')
    ax.set_xlabel('X Coordinate (m)')
    ax.set_ylabel('Y Coordinate (m)')
    ax.axis('equal') # Ensure equal scaling for x and y axes

    # Add a legend
    ax.legend()

    # Add a grid
    ax.grid(True, linestyle='--', alpha=0.6)

    plt.show()

# Example usage:
# Assuming you have a FiberHound instance named 'fiberhound' that has already run
# fiberhound = FiberHound() # Initialize
# fiberhound.run() # Run detection cycles to populate data
# visualize_fiberhound_data(fiberhound) # Visualize the data

NameError: name 'FiberHound' is not defined

Now you can run the `main()` function again in the previous code cell (`yUxrA7eyFxIl`) to simulate detections and populate the `fiberhound` instance with data. After that, run the code in the newly generated cell to visualize the results.